# <center>第4章 动态规划法</center>
&emsp;&emsp;在强化学习中，动态规划法（dynamic programming，DP）主要用于求解有模型的MDP问题。尽管在现实任务中难以获得完备的环境模型，且动态规划需要消耗大量的计算资源，但是作为强化学习的基础，动态规划法仍然具有非常重要的理论意义。事实上，所有其他强化学习方法，如蒙特卡罗法（Monte Carlo，MC）、时序差分法（Temporal Difference，TD）等，都是对动态规划法的一种近似，只是在学习过程中不再需要完整的环境模型，而且在计算资源的消耗方面也可以大大减少。<br>
&emsp;&emsp;动态规划算法主要包括基于模型的策略迭代（policy iteration）和基于模型的值迭代（value iteration）两种。这两种算法都是利用值函数来评价策略的，一旦计算出满足贝尔曼最优方程的最优状态值函数$v_*$或最优动作值函数$q_*$，就能得到最优策略。即利用$v_*$并借助环境模型，或直接利用$q_*$都可以得最优策略。
## 4.1 策略迭代
&emsp;&emsp;策略迭代通过构建策略的值函数（状态值函数$v_\pi$或动作值函数$q_\pi$）来评估当前策略，并利用这些值函数给出改进的新策略。策略迭代由策略评估（Policy Evaluation，PE）和策略改进（Policy Improvement，PI）两部分组成。<br>
&emsp;&emsp;策略评估：每一次策略评估都是一个迭代过程，对于一个给定的策略$\pi$，评估在该策略下，所有状态s（或状态-动作对$(s,a)$的值函数$v_\pi(s)$(或$q_\pi(s,a)$)。<br>
&emsp;&emsp;策略改进：在策略评估的基础上，直接利用（或状态值函数通过一步搜索计算得出的）策略$\pi$的动作值函数，然后通过贪心策略（或$\epsilon$-贪心策略）对策略进行改进。<br>
&emsp;&emsp;在采用某一策略$\pi$时，需要考虑该策略是否是最优策略。首先根据策略$\pi$的值函数$v_\pi$（或$q_\pi$）产生一个更优策略$\pi'$，再根据策略$\pi'$的值函数$v_{\pi'}'$（或$q_{\pi'}'$）得到一个更优策略$\pi''$，以此类推，通过这样的链式方法可以得到一个关于策略和值函数的更新序列，并且能够保证每一个新策略都比前一个策略更优（除非前一个策略已是最优策略）。在有限MDP中，策略有限，所以在多次迭代后，一定能收敛到最优策略和最优值函数。其链式关系为：<br>
<center><img src='./image/图4.1.jpg' width='500'></center><br>
其中，策略$\pi$的下标0，1，2，$...$表示迭代更新的次序。<br>

### 4.1.1策略评估
#### 4.1.1.1 基于状态值函数的策略评估
&emsp;&emsp;由第3章可知，对策略$\pi$进行评价就是估计在该策略下的状态值函数$v_\pi$，这在动态规划法中称为策略评估。在环境完备的情况下，状态值函数的贝尔曼方程（3.17）就是一个包含有$\mid{S}\mid$个未知数的$\mid{S}\mid$元方程组。理论上该方程组可以直接求解，但是当状态较多、甚至无穷多时，很难使用解方程组的方法来解决强化学习问题，因此引入迭代方法来简化求解过程。<br>
&emsp;&emsp;根据迭代思想，可以将状态值函数的贝尔曼方程（3.17）转化为迭代式（4.1），即基于状态值函数的策略评估迭代式：<br>
$$\begin{aligned}
v_\tau(s) &=\mathbb{E}_\pi(R_{t+1}+{\gamma}v_\pi(S_{t+1}|S_t=s))\\
&=\sum\limits_{a}\pi(a|s)\sum\limits_{s',r}p(s',r|s,a)[r+{\gamma}v_{\tau-1}(s')]
 \end{aligned} \tag{4.1}$$
&emsp;&emsp;初始值$v_0$可以为任意值（通常设为0）。在使用$v_\tau(s)$这样的迭代式时，默认采用同一策略$\pi$，为了突出迭代关系，省略$v_\pi(s)$的下标$\pi$。<br>
&emsp;&emsp;策略评估对每个状态$s$都采用相同的操作：根据给定的策略（动作分布），采取可能的动作，得到单步转移后的所有状态$s'$和奖励$r$，并利用下一状态$s'$的值函数$v_{\tau-1}(s')$，通过分布的期望值，更新状态$s$的值函数$v_\tau(s)$。该方法称为期望更新法（expected update）。另外，从式（4.1）可以看出，对状态$s$值函数的估算是建立在所有可能下一状态$s'$的值函数估计$v_{\tau-1}(s')$之上的，这种更新方式也称为自举方法（bootstrapping)。<br>
&emsp;&emsp;在保证$v_\pi$存在的前提下，当 $\tau→∞$时，序列{$v_\tau$}将会收敛$v_\pi$。在实际情况下，只有更新后的状态值基本不变，才可以停止迭代。通常可以采取以下两种方式提前结束迭代：<br>
&emsp;&emsp;（1）直接设置迭代次数。只要达到预期的迭代次数，即可停止迭代；<br>
&emsp;&emsp;（2）设定较小的阈值(次优界限)$\theta$。当$|v_\tau(s)-v_{\tau-1}(s)|_∞<\theta$时，停止迭代。比较两次迭代的状态值函数差的绝对值$\Delta$（或差的平方），当$\Delta$最大值小于阈值时，终止迭代。<br>
&emsp;&emsp;算法4.1给出了在有穷状态空间MDP中，基于状态值函数的策略评估算法。<br>

<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.1</b> 基于状态值函数的策略评估算法</font>
<br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
<font size=3.5>初始策略$\pi(a|s)$，动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in{S_s}$，初始化状态值函数，如$V(s)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实数值，如$\theta=0.01$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>策略评估：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略评估$\tau=1,2,\cdots$ </font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;$delta←0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;<b>for</b>&emsp;每个状态$s$&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;$v←V(s)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;
$V(s）←\sum\limits_{a}\pi(a|s)\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{V(s')}]$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;
$delta←max(delta,|v-V(s)|)$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.
<b>until</b> &emsp;$delta<\theta$   
</font><br>

<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
<font size=3.5>$v_\pi=V$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

&emsp;&emsp;算法4.1使用的迭代方式是异步计算方式，即在相邻的两个迭代轮次$\tau$和$\tau$-1，保存同一组状态值函数$V(s)$。在$V(s)$中，存储两轮混合的函数值。因此在每次计算中，如果状态$s$的值函数已被更新，那么当用到$V(s)$时，就使用已经更新过的数据。评估过程中，中间结果与状态评估的先后次序密切相关。另外一种迭代方式是同步计算方式，即在每一迭代轮次$\tau$，都保存相邻两轮的状态值函数：$V_\tau(s)$和$V_{\tau-1}(s)$。在计算$V_\tau(s)$过程中，使用的全部是上一轮的$V_{\tau-1}(s)$值。评估过程中，中间结果与状态评估的先后次序无关。在相同情况下，利用两种迭代方式评估，收敛后结果是相同的。但收敛速度方面相比较，通常异步计算方式收敛速度会更快。<br>
&emsp;&emsp;另外，算法4.1可用于有穷状态空间的确定MDP问题和随机MDP问题，这具体体现在MDP的状态转移动态$p(s',r|s,a)$上。对于确定MDP问题，在当前状态$s$下采取动作$a$，到达下一状态$s'$的概率为1，而到达其他状态的概率均为0。对于随机MDP问题，在当前状态$s$下采取动作$a$，到达下一状态$s'$是随机的。因此利用算法4.1解决确定MDP问题和随机MDP时，只是状态转移动态的变化，而算法本身不需要改变。这也体现出确定MDP是随机MDP的特例。<br>
&emsp;&emsp;下面利用算法4.1，分别来解决例3.1的确定环境扫地机器人任务和例3.2的随机环境扫地机器人任务。<br>
&emsp;&emsp;<b>例4.1</b> 对例3.1的确定环境扫地机器人任务进行策略评估。如图4.1所示，设定策略评估的起始位置为左下角，即充电桩位置，按序号顺序进行评估。机器人在非终止状态（除位置0、12、19）均采取等概率策略 $\pi(a|s)=1/|\mathcal{A}(s)|,a\subseteq{\mathcal{A}(s),|\mathcal{A}(s)|}$为当前状态$s$可采取的动作个数。扫地机器人最多可以采取{$Up,Down,Left,Right$}4个动作。到达充电桩位置时，$r$=+1；到达垃圾位置时，$r$=+3；撞到障碍物时，$r$=-10；其他情况，获得的奖赏$r$均为0。折扣系数$\gamma$=0.8，利用算法4.1计算，在策略$\pi$下，确定环境扫地机器人任务的状态值函数。<br>
<center><img src='./image/图4.2.png' width='250'></center><br>
<center>图4.1&ensp;扫地机器人环境</center>
&emsp;&emsp;根据算法4.1，在随机策略$\pi(a|s)$下，确定环境扫地机器人任务的评估过程如下：<br>
&emsp;&emsp;(1)当$\tau$=0时（$\tau$为迭代的次数），对于所有的$s$初始化为：$V(s)$=0；<br>
&emsp;&emsp;(2)当$\tau$=1时，以状态$S_{24}$为例。在策略$\pi$下，只能采取向下和向左2个动作，概率各为0.5。采取向下的动作时，到达状态$S_{19}$($V(S_{19})$=0)，并可以捡到垃圾，获得$r_1$=+3的奖赏；采取向左的动作时，到达状态$S_{23}$($V(S_{23}$)=-0.27)，获得$r_2$=0的奖赏。根据算法4.1计算得：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{24})&=\frac{1}{2}*1*(r_1+\gamma{V(S_{19})})+\frac{1}{2}*1*(r_2+\gamma{V(S_{23}}))\\
&=\frac{1}{2}*1*(3+0.8*0)+\frac{1}{2}*1*(0+0.8*(-0.27))\\
&\approx 1.39
 \end{aligned} $
<br>
&emsp;&emsp;同理可以计算状态$S_5、S_{17}$的状态值函数：
<br> 
&emsp;&emsp;$\begin{aligned}
V(S_{5})&=\frac{1}{3}*1*(r_1+\gamma{V(S_{10})})+\frac{1}{3}*1*(r_2+\gamma{V(S_{0}}))+\frac{1}{3}*1*(r_3+\gamma{V(S_{6})})\\
&=\frac{1}{3}*1*(0+0)+\frac{1}{3}*1*(1+0)+\frac{1}{3}*1*(0+0)\\
&\approx 0.33
 \end{aligned} $ 
<br>
&emsp;&emsp;$\begin{aligned}
V(S_{17})&=\frac{1}{4}*1*(r_1+\gamma{V(S_{22})})+\frac{1}{4}*1*(r_2+\gamma{V(S_{17}}))\\
&+\frac{1}{4}*1*(r_3+\gamma{V(S_{16})})+\frac{1}{4}*1*(r_4+\gamma{V(S_{18}}))\\
&=\frac{1}{4}*1*(0+0)+\frac{1}{4}*1*(-10+0)+\frac{1}{4}*1*(0+(-0.49))+\frac{1}{4}*1*(0+0)\\
&\approx -2.60
 \end{aligned} $
<br>
&emsp;&emsp;按顺序计算完一轮后，得到值函数 ，如图4.2（$\tau=1$）所示。<br>
&emsp;&emsp;（3）$\tau=2$ 时，仍然以状态$S_5$、$S_{17}$ 、$S_{24}$为例。采用异步计算方式，状态值函数的计算，通常与迭代计算顺序有关。这里策略评估的起始位置为左下角，即充电桩位置，按序号顺序进行评估。那么每一轮次中，这3个状态的计算顺序为$S_5$、$S_{17}$、$S_{24}$。3个状态值函数分别计算为：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{5})&=\frac{1}{3}*1*(r_1+\gamma{V(S_{10})})+\frac{1}{3}*1*(r_2+\gamma{V(S_{0}}))+\frac{1}{3}*1*(r_3+\gamma{V(S_{6})})\\
&=\frac{1}{3}*1*(0+0.8*0.09)+\frac{1}{3}*1*(0+0.8*0)+\frac{1}{3}*1*(0+0.8*0.13)\\
&\approx 0.39
\end{aligned} $
<br>
&emsp;&emsp;$\begin{aligned}
V(S_{17})&=\frac{1}{4}*1*(r_1+\gamma{V(S_{22})})+\frac{1}{4}*1*(r_2+\gamma{V(S_{17}}))\\
&+\frac{1}{4}*1*(r_3+\gamma{V(S_{16})})+\frac{1}{4}*1*(r_4+\gamma{V(S_{18}}))\\
&=\frac{1}{4}*1*(0+0.8*(-0.73))+\frac{1}{4}*1*(-10+0.8*(-2.60))\\
&+\frac{1}{4}*1*(0+0.8*(-1.27))+\frac{1}{4}*(0+0.8*(-2.29))\\
&\approx -3.48
 \end{aligned} $
<br>
&emsp;&emsp;$\begin{aligned}
V(S_{24})&=\frac{1}{2}*1*(r_1+\gamma{V(S_{19})})+\frac{1}{2}*1*(r_2+\gamma{V(S_{23}}))\\
&=\frac{1}{2}*1*(3+0.8*0)+\frac{1}{2}*1*(0+0.8*(-0.11)))\\
&= 1.46
 \end{aligned} $
<br>
&emsp;&emsp;按顺序计算完一轮后，得到状态值函数$V(s)$，如图4.2（$\tau=2$）所示。<br>
&emsp;&emsp;（4） 当$\tau=30$时，$|V_\tau(s)-V_{\tau-1}(s)|_∞<\theta$ ，认为$V_\tau(s)$已经收敛于$v_\pi(s)$，计算得到的$v_\pi(s)$就是在策略$\pi$下的有效评估。<br>
&emsp;&emsp;随着迭代的进行，每轮状态值函数的更新过程，如图4.2所示。<br>
<center><img src='./image/图4.3.jpg' width='1000'></center><br>
<center> 图4.2&ensp;面向确定环境扫地机器人任务的状态值函数$v_\pi$的评估过程（异步计算方式）</center>
&emsp;&emsp;对于例4.1的确定情况扫地机器人任务，策略评估时通常采用异步计算方式。而同步计算方式评估值函数时，环境状态越多，效率也越低。采用异步计算方式时，每一轮迭代都直接用新产生的值函数来替换旧的值函数，不需要对上一轮迭代的状态值函数进行备份，既减少了迭代次数，又节省了存储空间。对于该扫地机器人任务，采用异步计算方式进行评估，30轮迭代后既可以收敛到$v_\pi$，而采用同步计算方式，收敛到$v_\pi$则需要51轮迭代。另外，使用异步计算方式时，每次遍历并不需要对所有的状态值函数都做一次更新，而可以任意顺序更新状态值，这样其中的某些状态值可能会在其他状态值更新一次之前已经更新过多次。这种基于异步计算方式，以任意顺序进行规划的方法，称为异步动态规划法（Asynchronous Dynamic Progarmming，ADP）。ADP的特点归纳如下：<br>
&emsp;&emsp;（1）ADP可以对更新顺序进行调整，通常重要的状态优先更新；<br>
&emsp;&emsp;（2）在实际情况中，ADP必须保证完成所有状态的价值更新；<br>
&emsp;&emsp;（3）ADP并不一定能减少计算量。但该方法的作用在于：算法在改进策略之前不需要陷入无望的长时间扫描。<br>

In [1]:
import numpy as np
from book_gridword import GridWorldEnv

env = GridWorldEnv()
"""定义格子世界参数"""
world_h =  env.n_height
world_w = env.n_width
length = world_h * world_w
gamma = 0.8
state = [i for i in range(length)]  # 状态（编号）
action = ['n', 's', 'w', 'e']  # 动作名称
ds_action = {'n': -world_w, 'e': 1, 's': world_w, 'w': -1}
value = [0 for i in range(length)]  # 初始化状态值函数，均为0.  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
policy = np.zeros([length, len(action)])
suqe=[20, 21, 22, 23, 24, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 5, 6, 7, 8, 9, 0, 1, 2, 3,
                  4]

In [2]:
# 定义奖励
def reward(s):
    if s == 20:  # 到充电站
        return 1
    elif s == 12:  # 到陷阱中
        return -10
    elif s == 9:  # 到垃圾处
        return 3
    else:
        return 0  # 其他
    # in表示0是[*，*，*]中的一个


def getAction(a):
    if a == 'n':
        return 0
    elif a == 'e':
        return 3
    elif a == 's':
        return 1
    elif a == 'w':
        return 2


# 在s状态下执行动作a，返回下一状态（编号）
def next_states(s, a):
    # 越过边界时pass
    if (s < world_w and a == 'n') \
            or (s % world_w == 0 and a == 'w') \
            or (s > length - world_w - 1 and a == 's') \
            or ((s + 1) % world_w == 0 and a == 'e'):  # (s % (world_w - 1) == 0 and a == 'e' and s != 0)
        next_state = s  # 表现为next_state不变
    else:
        next_state = s + ds_action[a]  # 进入下一个状态
    return next_state


# 在s状态下执行动作，返回所有可能的下一状态（编号）list
def getsuccessor(s):
    successor = []
    for a in action:  # 遍历四个动作
        if s == next_states(s, a):
            continue
        else:
            # print("状态s=%s,动作a=%s"%(s,a))
            next = next_states(s, a)  # 得到下一个状态（编号）
        successor.append(next)  # 以list保存当前状态s下执行四个动作的下一状态
    # print(len(successor))
    return successor


def initPolicy():
    for s in range(length):
        for a in action:
            if next_states(s, a) == s:
                continue
            newAction = getAction(a)
            policy[s][newAction] = 1 / len(getsuccessor(s))
    # print(policy)


In [3]:
def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    iter = 0

    while True:
        k = -1
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in suqe:  # 遍历所有状态 [0~25]
            k += 1
            if s in [9, 20, 12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            print("第%d 的状态" % (k), end="")
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s, a)
                rewards = reward(next_state)
                if next_state == 12:
                    v += policy[s][newAction] * (rewards + gamma * V[s])
                    # print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
                    print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
                else:
                    v += policy[s][newAction] * (rewards + gamma * V[next_state])
                    #     print("%.2f*(%d+%.1f*%.2f)+" % (policy[s][newAction], rewards, gamma, value[next_state]), end="")
                    # print()
                    # successor = getsuccessor(s)
                    # for next_state in successor:
                    #     rewards = reward(next_state)
                    #     v += 1 / len(successor) * (rewards + gamma * V[next_state])
                    print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
            print("v = %.3f" % (v))

            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            V[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
        value = np.array(V).reshape(world_h, world_w)
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(np.round(value, decimals=2))
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
    return V  # 一轮迭代结束后，状态值函数暂时固定


In [4]:

initPolicy()
value = policy_eval()

第1 的状态 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(1+0.8*0.000)+ 0.33*(0+0.8*0.000)+v = 0.333
第2 的状态 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.333)+ 0.33*(0+0.8*0.000)+v = 0.089
第3 的状态 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.089)+ 0.33*(0+0.8*0.000)+v = 0.024
第4 的状态 0.50*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.50*(0+0.8*0.024)+ 0.00*(0+0.8*0.000)+v = 0.009
第5 的状态 0.33*(0+0.8*0.000)+ 0.33*(1+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+v = 0.333
第6 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.333)+ 0.25*(0+0.8*0.333)+ 0.25*(0+0.8*0.000)+v = 0.133
第7 的状态 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*0.089)+ 0.25*(0+0.8*0.133)+ 0.25*(0+0.8*0.000)+v = -2.456
第8 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.024)+ 0.25*(0+0.8*-2.456)+ 0.25*(0+0.8*0.000)+v = -0.486
第9 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.009)+ 0.33*(0+0.8*-0.486)+ 0.00*(0+0.8*0.000)+v = -0.127
第10 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.333)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+v = 0.089
第11 的状态 0.25*(0+0.8*0.000)+ 0.25

第7 的状态 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*-1.769)+ 0.25*(0+0.8*-2.159)+ 0.25*(0+0.8*-2.157)+v = -4.646
第8 的状态 0.25*(0+0.8*-3.984)+ 0.25*(0+0.8*-1.277)+ 0.25*(0+0.8*-4.646)+ 0.25*(0+0.8*-0.885)+v = -2.158
第9 的状态 0.33*(0+0.8*-0.298)+ 0.33*(0+0.8*-0.865)+ 0.33*(0+0.8*-2.158)+ 0.00*(0+0.8*-0.885)+v = -0.886
第10 的状态 0.33*(0+0.8*-1.411)+ 0.33*(0+0.8*-0.728)+ 0.00*(0+0.8*-1.824)+ 0.33*(0+0.8*-4.711)+v = -1.827
第11 的状态 0.25*(0+0.8*-2.368)+ 0.25*(0+0.8*-2.159)+ 0.25*(0+0.8*-1.827)+ 0.25*(-10+0.8*0.000)+v = -4.713
第13 的状态 0.25*(0+0.8*-0.985)+ 0.25*(0+0.8*-2.158)+ 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*-0.298)+v = -3.985
第14 的状态 0.33*(3+0.8*0.000)+ 0.33*(0+0.8*-0.886)+ 0.33*(0+0.8*-3.985)+ 0.00*(0+0.8*-0.298)+v = -0.299
第15 的状态 0.33*(0+0.8*-1.106)+ 0.33*(0+0.8*-1.827)+ 0.00*(0+0.8*-1.411)+ 0.33*(0+0.8*-2.368)+v = -1.414
第16 的状态 0.25*(0+0.8*-1.356)+ 0.25*(0+0.8*-4.713)+ 0.25*(0+0.8*-1.414)+ 0.25*(0+0.8*-4.365)+v = -2.370
第17 的状态 0.25*(0+0.8*-1.613)+ 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*-2.370)+ 0.25*(0+0.8

第9 的状态 0.33*(0+0.8*-0.300)+ 0.33*(0+0.8*-0.867)+ 0.33*(0+0.8*-2.160)+ 0.00*(0+0.8*-0.887)+v = -0.887
第10 的状态 0.33*(0+0.8*-1.417)+ 0.33*(0+0.8*-0.731)+ 0.00*(0+0.8*-1.831)+ 0.33*(0+0.8*-4.716)+v = -1.831
第11 的状态 0.25*(0+0.8*-2.372)+ 0.25*(0+0.8*-2.162)+ 0.25*(0+0.8*-1.831)+ 0.25*(-10+0.8*0.000)+v = -4.716
第13 的状态 0.25*(0+0.8*-0.987)+ 0.25*(0+0.8*-2.160)+ 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*-0.300)+v = -3.987
第14 的状态 0.33*(3+0.8*0.000)+ 0.33*(0+0.8*-0.887)+ 0.33*(0+0.8*-3.987)+ 0.00*(0+0.8*-0.300)+v = -0.300
第15 的状态 0.33*(0+0.8*-1.111)+ 0.33*(0+0.8*-1.831)+ 0.00*(0+0.8*-1.417)+ 0.33*(0+0.8*-2.372)+v = -1.417
第16 的状态 0.25*(0+0.8*-1.359)+ 0.25*(0+0.8*-4.716)+ 0.25*(0+0.8*-1.417)+ 0.25*(0+0.8*-4.369)+v = -2.372
第17 的状态 0.25*(0+0.8*-1.615)+ 0.25*(-10+0.8*0.000)+ 0.25*(0+0.8*-2.372)+ 0.25*(0+0.8*-0.987)+v = -4.369
第18 的状态 0.25*(0+0.8*-0.329)+ 0.25*(0+0.8*-3.987)+ 0.25*(0+0.8*-4.369)+ 0.25*(3+0.8*0.000)+v = -0.987
第20 的状态 0.00*(0+0.8*-1.111)+ 0.50*(0+0.8*-1.417)+ 0.00*(0+0.8*-1.111)+ 0.50*(0+0.8

&emsp;&emsp;<b>例4.2</b> 对例3.2的随机环境下扫地机器人任务进行策略评估。重新考虑例3.1的扫地机器人问题。假设由于地面的问题，采取某一动作后，状态转换不再确定。当采取某一动作试图向某一方向移动时，机器人成功移动的概率为0.80，保持原地不动的概率为0.15，移动到相反方向的概率为0.05，具体如图4.3所示，其中$s'$表示下一个状态，$s''$表示相反方向的状态。<br>
<center><img src="./image/图4.4.jpg" wight ="250"></center><br>
<center> 图4.3&ensp;随机环境扫地机器人状态转移示意图</center>
&emsp;&emsp;在其他条件与例4.1不变的情况下，采用异步计算方式，给出策略$\pi$下的随机环境扫地机器人任务的状态值函数更新过程。<br>
&emsp;&emsp;根据算法4.1，在随机策略$\pi(a|s)$下，随机环境扫地机器人任务的评估过程如下：<br>
&emsp;&emsp;（1）当$\tau$=0时（$\tau$是迭代的次数），对于所有的$s$初始化为：$V(s)$=0；<br>
&emsp;&emsp;（2）当$\tau$=1时，以状态$S_{24}$为例。在策略$\pi$下，只能采取向下和向左2个动作，概率各为0.5。采取向下的动作时，以0.8的概率到达状态$S_{19}$($V(S_{19}$)=0)，并可以捡到垃圾，获得$r_{11}$=+3的奖赏；以0.15的概率保持原地$S_{24}$($V(S_{24}$)=0)不动，获得$r_{12}$=0的奖赏；以0.05的概率采取冲出边界的动作，保持原地$S_{24}$($V(S_{24}$)=0)不动，获得$r_{13}$=0的奖赏。采取向左的动作时，以0.8的概率到达状态$S_{23}$($V(S_{23}$=-0.132))，获得$r_{21}$=0的奖赏；以0.15的概率保持原地$S_{24}$($V(S_{24}$)=0)不动，获得$r_{22}$=0的奖赏；以0.05的概率采取冲出边界的动作，保持原地$S_{24}$($V(S_{24}$)=0)不动，获得$r_{23}$=0的奖赏。根据算法4.1计算得：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{24})&=\frac{1}{2}*[0.8*(r_{11}+\gamma{V(S_{19})})+0.15*(r_{12}+\gamma{V(S_{24})})+0.05*(r_{13}+\gamma{V(S_{24})})]\\
&+\frac{1}{2}*[0.8*(r_{21}+\gamma{V(S_{23})})+0.15*(r_{22}+\gamma{V(S_{24})})+0.05*(r_{23}+\gamma{V(S_{24})})]\\
&=\frac{1}{2}*[0.8*(3+0.8*0)+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&+\frac{1}{2}*[0.8*(0+0.8*(-0.132))+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&\approx 1.16
\end{aligned} $
<br>
<br>
&emsp;&emsp;同理可以计算状态$S_5$、$S_{17}$的状态值函数：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{5})&=\frac{1}{3}*[0.8*(r_{11}+\gamma{V(S_{10})})+0.15*(r_{12}+\gamma{V(S_{5})})+0.05*(r_{13}+\gamma{V(S_{0})})]\\
&+\frac{1}{3}*[0.8*(r_{21}+\gamma{V(S_{0})})+0.15*(r_{22}+\gamma{V(S_{5})})+0.05*(r_{23}+\gamma{V(S_{24})})]\\
&+\frac{1}{3}*[0.8*(r_{31}+\gamma{V(S_{6})})+0.15*(r_{32}+\gamma{V(S_{5})})+0.05*(r_{33}+\gamma{V(S_{5})})]\\
&=\frac{1}{3}*[0.8*(0+0.8*0)+0.15*(0+0.8*0)+0.05*(1+0.8*0)]\\
&+\frac{1}{3}*[0.8*(0+0.8*0)+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&+\frac{1}{3}*[0.8*(0+0.8*0)+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&\approx 0.28
 \end{aligned} $
<br>

&emsp;&emsp;$\begin{aligned}
V(S_{17})&=\frac{1}{4}*[0.8*(r_{11}+\gamma{V(S_{22})})+0.15*(r_{12}+\gamma{V(S_{17})})+0.05*(r_{13}+\gamma{V(S_{17})})]\\
&+\frac{1}{4}*[0.8*(r_{21}+\gamma{V(S_{17})})+0.15*(r_{22}+\gamma{V(S_{17})})+0.05*(r_{23}+\gamma{V(S_{22})})]\\
&+\frac{1}{4}*[0.8*(r_{31}+\gamma{V(S_{16})})+0.15*(r_{32}+\gamma{V(S_{17})})+0.05*(r_{33}+\gamma{V(S_{18})})]\\
&+\frac{1}{4}*[0.8*(r_{41}+\gamma{V(S_{18})})+0.15*(r_{42}+\gamma{V(S_{17})})+0.05*(r_{43}+\gamma{V(S_{16})})]\\
&=\frac{1}{4}*[0.8*(0+0.8*0)+0.15*(0+0.8*0)+0.05*(-10+0.8*0)]\\
&+\frac{1}{4}*[0.8*(-10+0.8*0)+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&+\frac{1}{4}*[0.8*(0+0.8*(-0.354))+0.15*(0+0.8*0)+0.05*(0+0.8*0)]\\
&+\frac{1}{4}*[0.8*(0+0.8*0)+0.15*(0+0.8*0)+0.05*(0+0.8*(-0.354))]\\
&\approx -2.19
 \end{aligned} $
<br>
 &emsp;&emsp;按顺序计算完一轮后，得到值函数$V(s)$，如图4.4（$\tau$=1）所示。<br>
 <br>
 &emsp;&emsp;（3）当$\tau$=2时，仍然以状态$S_5$、$S_{17}$、$S_{24}$为例。在每一轮次中，这3个状态的计算顺序为$S_5$、$S_{17}$、$S_{24}$。3个状态值函数分别计算为：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{5})&=\frac{1}{3}*[0.8*(r_{11}+\gamma{V(S_{10})})+0.15*(r_{12}+\gamma{V(S_{5})})+0.05*(r_{13}+\gamma{V(S_{0})})]\\
&+\frac{1}{3}*[0.8*(r_{21}+\gamma{V(S_{0})})+0.15*(r_{22}+\gamma{V(S_{5})})+0.05*(r_{23}+\gamma{V(S_{24})})]\\
&+\frac{1}{3}*[0.8*(r_{31}+\gamma{V(S_{6})})+0.15*(r_{32}+\gamma{V(S_{5})})+0.05*(r_{33}+\gamma{V(S_{5})})]\\
&=\frac{1}{3}*[0.8*(0+0.8*0.064)+0.15*(0+0.8*0.283)+0.05*(1+0.8*0)]\\
&+\frac{1}{3}*[0.8*(1+0.8*0)+0.15*(0+0.8*0.283)+0.05*(0+0.8*0.064)]\\
&+\frac{1}{3}*[0.8*(0+0.8*0.096)+0.15*(0+0.8*0.283)+0.05*(0+0.8*0.283)]\\
&\approx 0.36
\end{aligned} $
<br>

&emsp;&emsp;$\begin{aligned}
V(S_{17})&=\frac{1}{4}*[0.8*(r_{11}+\gamma{V(S_{22})})+0.15*(r_{12}+\gamma{V(S_{17})})+0.05*(r_{13}+\gamma{V(S_{17})})]\\
&+\frac{1}{4}*[0.8*(r_{21}+\gamma{V(S_{17})})+0.15*(r_{22}+\gamma{V(S_{17})})+0.05*(r_{23}+\gamma{V(S_{22})})]\\
&+\frac{1}{4}*[0.8*(r_{31}+\gamma{V(S_{16})})+0.15*(r_{32}+\gamma{V(S_{17})})+0.05*(r_{33}+\gamma{V(S_{18})})]\\
&+\frac{1}{4}*[0.8*(r_{41}+\gamma{V(S_{18})})+0.15*(r_{42}+\gamma{V(S_{17})})+0.05*(r_{43}+\gamma{V(S_{16})})]\\
&=\frac{1}{4}*[0.8*(0+0.8*(-0.483))+0.15*(0+0.8*(-2.185))+0.05*(-10+0.8*0)]\\
&+\frac{1}{4}*[0.8*(-10+0.8*(-2.185))+0.15*(0+0.8*(-2.185))+0.05*(0+0.8*(-0.483))]\\
&+\frac{1}{4}*[0.8*(0+0.8*(-0.951))+0.15*(0+0.8*-2.185)+0.05*(0+0.8*(-0.106))]\\
&+\frac{1}{4}*[0.8*(0+0.8*(-0.106))+0.15*(0+0.8*(-2.185))+0.05*(0+0.8*(-0.951))]\\
&\approx -3.00
\end{aligned} $
<br>

&emsp;&emsp;$\begin{aligned}
V(S_{24})&=\frac{1}{2}*[0.8*(r_{11}+\gamma{V(S_{19})})+0.15*(r_{12}+\gamma{V(S_{24})})+0.05*(r_{13}+\gamma{V(S_{24})})]\\
&+\frac{1}{2}*[0.8*(r_{21}+\gamma{V(S_{23})})+0.15*(r_{22}+\gamma{V(S_{24})})+0.05*(r_{23}+\gamma{V(S_{24})})]\\
&=\frac{1}{2}*[0.8*(3+0.8*0)+0.15*(0+0.8*1.158)+0.05*(0+0.8*1.158)]\\
&+\frac{1}{2}*[0.8*(0+0.8*(-0.023))+0.15*(0+0.8*1.58)+0.05*(0+0.8*1.158)]\\
&\approx 1.38
\end{aligned} $
<br>
<br>
&emsp;&emsp;按顺序计算完一轮后，得到状态值函数$V(s)$，如图4.4（$\tau=2$）所示。<br>
&emsp;&emsp;（4） 当 时$\tau=34$，$|V_\tau(s)-V_{\tau-1}|_∞<\theta$，认为$V_\tau(s)$已经收敛于$V_\pi(s)$，计算得到的$V_\pi(s)$就是在策略$\pi$下的有效评估。<br>
&emsp;&emsp;随着迭代的进行，每轮状态值函数更新，如图4.4所示。<br>
<center><img src="./image/图4.5.jpg" wight ="1000"></center><br>
<center> 图4.4&ensp;面向随机环境扫地机器人任务的状态值函数$v_\pi$的评估过程（异步计算方式）</center>

In [5]:
def envActionPolicy(a):
    if a == 'n':
        return 's'
    elif a == 's':
        return 'n'
    elif a == 'e':
        return 'w'
    elif a == 'w':
        return 'e'
    
def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    iter = 0

    while True:
        k = -1
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in suqe:  # 遍历所有状态 [0~25]
            k += 1
            if s in [9, 20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            print("第%d 的状态"%(k),end="")
            for a in action:
                newAction = getAction(a)
                envAction = envActionPolicy(a)
                env_state = next_states(s,envAction)
                next_state = next_states(s,a)
                rewards = reward(next_state)
                env_rewards = reward(env_state)
                if next_state == 12:
                    v += policy[s][newAction] * (0.8*(rewards + gamma * V[s])+0.15*(gamma * V[s])+0.05*(env_rewards + gamma*V[env_state]))
                    print(" %.2f*(0.8*（%d+%.1f*%.3f)+0.15*（%.f）+0.05*(%.2f+%.2f))+" % (
                        policy[s][newAction], rewards, gamma, V[s], V[s], env_rewards,V[env_state]), end="")
                else:
                    v += policy[s][newAction] * (0.8*(rewards + gamma * V[next_state])+0.15*(gamma * V[s])+0.05*(env_rewards + gamma*V[env_state]))
                    print(" %.2f*(0.8*（%d+%.1f*%.3f)+0.15*（%.f）+0.05*(%.2f+%.2f))+" % (
                        policy[s][newAction], rewards, gamma, V[next_state],V[s],env_rewards,V[env_state]), end="")
            print("v = %.3f"%(v))

            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            V[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
        value = np.array(V).reshape(world_h, world_w)
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(np.round(value, decimals=2))
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
    return V  # 一轮迭代结束后，状态值函数暂时固定
initPolicy()
value = policy_eval()

第1 的状态 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.00*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（1+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(1.00+0.00))+v = 0.283
第2 的状态 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.00*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（0+0.8*0.283)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.28))+v = 0.064
第3 的状态 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.00*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（0+0.8*0.064)+0.15*（0）+0.05*(0.00+0.00))+ 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.06))+v = 0.015
第4 的状态 0.50*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.00*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.00))+ 0.50*(0.8*（0+0.8*0.015)+0.15*（0）+0.05*(0.00+0.00))+ 0.00*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(0.00+0.01))+v = 0.005
第5 的状态 0.33*(0.8*（0+0.8*0.000)+0.15*（0）+0.05*(1.00+0.00))+ 0.33*(0.8*（1+0.8*0.000)+0.15*（0）+0.05*(0.00+0

第9 的状态 0.33*(0.8*（0+0.8*-0.042)+0.15*（-1）+0.05*(0.00+-0.57))+ 0.33*(0.8*（0+0.8*-0.574)+0.15*（-1）+0.05*(0.00+-0.04))+ 0.33*(0.8*（0+0.8*-1.765)+0.15*（-1）+0.05*(0.00+-0.58))+ 0.00*(0.8*（0+0.8*-0.584)+0.15*（-1）+0.05*(0.00+-1.76))+v = -0.594
第10 的状态 0.33*(0.8*（0+0.8*-1.008)+0.15*（-1）+0.05*(0.00+-0.46))+ 0.33*(0.8*（0+0.8*-0.457)+0.15*（-1）+0.05*(0.00+-1.01))+ 0.00*(0.8*（0+0.8*-1.372)+0.15*（-1）+0.05*(0.00+-4.12))+ 0.33*(0.8*（0+0.8*-4.120)+0.15*（-1）+0.05*(0.00+-1.37))+v = -1.394
第11 的状态 0.25*(0.8*（0+0.8*-1.916)+0.15*（-4）+0.05*(0.00+-1.76))+ 0.25*(0.8*（0+0.8*-1.755)+0.15*（-4）+0.05*(0.00+-1.92))+ 0.25*(0.8*（0+0.8*-1.394)+0.15*（-4）+0.05*(-10.00+0.00))+ 0.25*(0.8*（-10+0.8*-4.120)+0.15*（-4）+0.05*(0.00+-1.39))+v = -4.140
第13 的状态 0.25*(0.8*（0+0.8*-0.729)+0.15*（-4）+0.05*(0.00+-1.76))+ 0.25*(0.8*（0+0.8*-1.765)+0.15*（-4）+0.05*(0.00+-0.73))+ 0.25*(0.8*（-10+0.8*-3.538)+0.15*（-4）+0.05*(0.00+-0.04))+ 0.25*(0.8*（0+0.8*-0.042)+0.15*（-4）+0.05*(-10.00+0.00))+v = -3.547
第14 的状态 0.33*(0.8*（3+0.8*0.000)+0.15*（-0）+0

第4 的状态 0.50*(0.8*（0+0.8*-0.609)+0.15*（-1）+0.05*(0.00+-0.59))+ 0.00*(0.8*（0+0.8*-0.595)+0.15*（-1）+0.05*(0.00+-0.61))+ 0.50*(0.8*（0+0.8*-0.958)+0.15*（-1）+0.05*(0.00+-0.59))+ 0.00*(0.8*（0+0.8*-0.595)+0.15*（-1）+0.05*(0.00+-0.96))+v = -0.596
第5 的状态 0.33*(0.8*（0+0.8*-1.427)+0.15*（-0）+0.05*(1.00+0.00))+ 0.33*(0.8*（1+0.8*0.000)+0.15*（-0）+0.05*(0.00+-1.43))+ 0.00*(0.8*（0+0.8*-0.483)+0.15*（-0）+0.05*(0.00+-1.78))+ 0.33*(0.8*（0+0.8*-1.784)+0.15*（-0）+0.05*(0.00+-0.48))+v = -0.485
第6 的状态 0.25*(0.8*（0+0.8*-4.169)+0.15*（-2）+0.05*(0.00+-0.47))+ 0.25*(0.8*（0+0.8*-0.474)+0.15*（-2）+0.05*(0.00+-4.17))+ 0.25*(0.8*（0+0.8*-0.485)+0.15*（-2）+0.05*(0.00+-4.12))+ 0.25*(0.8*（0+0.8*-4.119)+0.15*（-2）+0.05*(0.00+-0.49))+v = -1.786
第7 的状态 0.25*(0.8*（-10+0.8*-4.119)+0.15*（-4）+0.05*(0.00+-1.39))+ 0.25*(0.8*（0+0.8*-1.388)+0.15*（-4）+0.05*(-10.00+0.00))+ 0.25*(0.8*（0+0.8*-1.786)+0.15*（-4）+0.05*(0.00+-1.78))+ 0.25*(0.8*（0+0.8*-1.785)+0.15*（-4）+0.05*(0.00+-1.79))+v = -4.121
第8 的状态 0.25*(0.8*（0+0.8*-3.560)+0.15*（-2）+0.05*(0.0

k= 25
[[-0.8  -1.02 -1.27 -0.16  1.37]
 [-1.07 -1.96 -3.89 -0.75  0.  ]
 [-1.43 -4.18  0.   -3.56 -0.06]
 [-0.49 -1.79 -4.13 -1.79 -0.61]
 [ 0.   -0.48 -1.39 -0.96 -0.6 ]]
第1 的状态 0.33*(0.8*（0+0.8*-1.790)+0.15*（-0）+0.05*(0.00+-0.48))+ 0.00*(0.8*（0+0.8*-0.478)+0.15*（-0）+0.05*(0.00+-1.79))+ 0.33*(0.8*（1+0.8*0.000)+0.15*（-0）+0.05*(0.00+-1.39))+ 0.33*(0.8*（0+0.8*-1.392)+0.15*（-0）+0.05*(1.00+0.00))+v = -0.478
第2 的状态 0.33*(0.8*（0+0.8*-4.125)+0.15*（-1）+0.05*(0.00+-1.39))+ 0.00*(0.8*（0+0.8*-1.392)+0.15*（-1）+0.05*(0.00+-4.13))+ 0.33*(0.8*（0+0.8*-0.478)+0.15*（-1）+0.05*(0.00+-0.96))+ 0.33*(0.8*（0+0.8*-0.961)+0.15*（-1）+0.05*(0.00+-0.48))+v = -1.392
第3 的状态 0.33*(0.8*（0+0.8*-1.789)+0.15*（-1）+0.05*(0.00+-0.96))+ 0.00*(0.8*（0+0.8*-0.961)+0.15*（-1）+0.05*(0.00+-1.79))+ 0.33*(0.8*（0+0.8*-1.392)+0.15*（-1）+0.05*(0.00+-0.60))+ 0.33*(0.8*（0+0.8*-0.599)+0.15*（-1）+0.05*(0.00+-1.39))+v = -0.961
第4 的状态 0.50*(0.8*（0+0.8*-0.612)+0.15*（-1）+0.05*(0.00+-0.60))+ 0.00*(0.8*（0+0.8*-0.599)+0.15*（-1）+0.05*(0.00+-0.61))+ 0.

第24 的状态 0.00*(0.8*（0+0.8*1.369)+0.15*（1）+0.05*(3.00+0.00))+ 0.50*(0.8*（3+0.8*0.000)+0.15*（1）+0.05*(0.00+1.37))+ 0.50*(0.8*（0+0.8*-0.156)+0.15*（1）+0.05*(0.00+1.37))+ 0.00*(0.8*（0+0.8*1.369)+0.15*（1）+0.05*(0.00+-0.16))+v = 1.369
k= 29
[[-0.8  -1.02 -1.27 -0.16  1.37]
 [-1.07 -1.96 -3.89 -0.75  0.  ]
 [-1.43 -4.18  0.   -3.56 -0.06]
 [-0.49 -1.79 -4.13 -1.79 -0.61]
 [ 0.   -0.48 -1.39 -0.96 -0.6 ]]
第1 的状态 0.33*(0.8*（0+0.8*-1.790)+0.15*（-0）+0.05*(0.00+-0.48))+ 0.00*(0.8*（0+0.8*-0.478)+0.15*（-0）+0.05*(0.00+-1.79))+ 0.33*(0.8*（1+0.8*0.000)+0.15*（-0）+0.05*(0.00+-1.39))+ 0.33*(0.8*（0+0.8*-1.392)+0.15*（-0）+0.05*(1.00+0.00))+v = -0.478
第2 的状态 0.33*(0.8*（0+0.8*-4.125)+0.15*（-1）+0.05*(0.00+-1.39))+ 0.00*(0.8*（0+0.8*-1.392)+0.15*（-1）+0.05*(0.00+-4.13))+ 0.33*(0.8*（0+0.8*-0.478)+0.15*（-1）+0.05*(0.00+-0.96))+ 0.33*(0.8*（0+0.8*-0.961)+0.15*（-1）+0.05*(0.00+-0.48))+v = -1.392
第3 的状态 0.33*(0.8*（0+0.8*-1.789)+0.15*（-1）+0.05*(0.00+-0.96))+ 0.00*(0.8*（0+0.8*-0.961)+0.15*（-1）+0.05*(0.00+-1.79))+ 0.33*(0.8*（0

第11 的状态 0.25*(0.8*（0+0.8*-1.961)+0.15*（-4）+0.05*(0.00+-1.79))+ 0.25*(0.8*（0+0.8*-1.790)+0.15*（-4）+0.05*(0.00+-1.96))+ 0.25*(0.8*（0+0.8*-1.435)+0.15*（-4）+0.05*(-10.00+0.00))+ 0.25*(0.8*（-10+0.8*-4.176)+0.15*（-4）+0.05*(0.00+-1.43))+v = -4.176
第13 的状态 0.25*(0.8*（0+0.8*-0.745)+0.15*（-4）+0.05*(0.00+-1.79))+ 0.25*(0.8*（0+0.8*-1.789)+0.15*（-4）+0.05*(0.00+-0.75))+ 0.25*(0.8*（-10+0.8*-3.563)+0.15*（-4）+0.05*(0.00+-0.06))+ 0.25*(0.8*（0+0.8*-0.056)+0.15*（-4）+0.05*(-10.00+0.00))+v = -3.563
第14 的状态 0.33*(0.8*（3+0.8*0.000)+0.15*（-0）+0.05*(0.00+-0.61))+ 0.33*(0.8*（0+0.8*-0.612)+0.15*（-0）+0.05*(3.00+0.00))+ 0.33*(0.8*（0+0.8*-3.563)+0.15*（-0）+0.05*(0.00+-0.06))+ 0.00*(0.8*（0+0.8*-0.056)+0.15*（-0）+0.05*(0.00+-3.56))+v = -0.056
第15 的状态 0.33*(0.8*（0+0.8*-0.795)+0.15*（-1）+0.05*(0.00+-1.43))+ 0.33*(0.8*（0+0.8*-1.435)+0.15*（-1）+0.05*(0.00+-0.80))+ 0.00*(0.8*（0+0.8*-1.066)+0.15*（-1）+0.05*(0.00+-1.96))+ 0.33*(0.8*（0+0.8*-1.961)+0.15*（-1）+0.05*(0.00+-1.07))+v = -1.066
第16 的状态 0.25*(0.8*（0+0.8*-1.022)+0.15*（-2）+0

#### 4.1.1.2 基于动作值函数的策略评估
&emsp;&emsp;与基于状态值函数的策略评估一样，根据迭代思想，可以将动作值函数的贝尔曼方程（3.19）转化为迭代式（4.2），即基于动作值（$Q$值）函数的策略评估迭代式：<br>
$$\begin{aligned}
q_\tau(s,a) &= \mathbb{E}_\pi(G_t|S_t=s,A_t=a)\\
&=\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{\sum\limits_{a'}\pi(a'|s')q_{\tau-1}(s',a')}]
 \end{aligned} \tag{4.2}$$
&emsp;&emsp;使用$Q$值函数的策略评估，在当前策略$\pi$下，从任意$Q$值函数$q_0$开始，在每一轮迭代中，利用式（4.2）来更新$Q$值函数。<br>
<br>
&emsp;&emsp;算法4.2给出了在有穷状态空间MDP中，基于动作值函数的策略评估算法。<br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.2</b> 基于动作值函数的策略评估算法</font>
<br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
&emsp;&emsp;<font size=3.5>初始策略$\pi(a|s)$，动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in{S_s}$，$a\in{\mathcal{A(s)}}$，初始化动作值函数，如$Q(s,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实数值，如$\theta$=0.01</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>策略评估：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略评估$\tau=1,2,\cdots$ </font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;$delta←0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;<b>for</b>&emsp;每个状态-动作对$(s,a)$&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;$q←Q(s,a)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;
$Q(s,a)←\sum\limits_{s',r} p(s',r|s,a)[r+\gamma\sum\limits_{a'}(\pi(a'|s')Q(s',a'))]$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;
$delta←max(delta,|q-Q(s,a)|)$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.
<b>until</b> &emsp;$delta<\theta$   
</font><br>

<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;&emsp;<font size=3.5>$q_*=Q$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>
&emsp;&emsp;<b>例4.3</b> 利用算法4.2，基于$Q$值函数对确定环境扫地机器人任务进行策略评估。如图4.1所示，设定策略评估的起始位置为左下角，即充电桩位置，按序号顺序进行评估。动作按$Up$,$Down$,$Left$,$Right$顺序评估。折扣系数$\gamma$=0.8，在等概率策略$\pi$下，计算确定环境扫地机器人任务的动作值函数。<br>
&emsp;&emsp;根据算法4.2，在随机策略$\pi(a|s)$下，确定环境扫地机器人任务的评估过程如下：<br>
&emsp;&emsp;（1）当$\tau$=0时（$\tau$为迭代的次数），对于所有的($s$,$a$)初始化为：$Q(s,a)$=0；<br>
&emsp;&emsp;（2）当$\tau$=1时，以状态$S_7$的4个动作{$Up$,$Down$,$Left$,$Right$}为例。根据扫地机器人的确定环境MDP模型，$(S_7,Up)$遇到障碍物，保持原地$S_7$不动，得到-10的惩罚，状态$S_7$中可以采取{$Up$,$Down$,$Left$,$Right$}4个动作，概率各为$\frac{1}{4}$；$(S_7,Down)$到达状态$S_2$，得到0的奖赏，状态$S_2$中可以采取{$Up$,$Left$,$Right$}3个动作，概率各为$\frac{1}{3}$；$(S_7,Left)$到达状态$S_6$，得到0的奖赏，状态$S_6$中可以采取{$Up$,$Down$,$Left$,$Right$}4个动作，概率各为$\frac{1}{4}$；$(S_7,Right)$到达状态$S_8$，得到0的奖赏，状态$S_8$中可以采取{$Up$,$Down$,$Left$,$Right$}4个动作，概率各为$\frac{1}{4}$。终上所述，根据算法4.2计算得：<br>
&emsp;&emsp;$\begin{aligned}
Q(S_7,Up) &=r+\gamma\sum\limits_{a'}(\pi(a'|S_7)Q(S_7,a'))\\
&=-10+0.8*[\frac{1}{4}*Q(S_7,Up)+\frac{1}{4}*Q(S_7,Down)+\frac{1}{4}*Q(S_7,Left)+\frac{1}{4}*Q(S_7,Right)];\\
&=-10+0.8*(0.25*0.25*0+0.25*0.25*0+0.25*0+0.25*0)\\
&=-10.00\\
\end{aligned} $
<br>
&emsp;&emsp;$\begin{aligned}
Q(S_7,Down) &=r+\gamma\sum\limits_{a'}(\pi(a'|S_2)Q(S_2,a'))\\
          &=0+0.8*[\frac{1}{3}*Q(S_2,Up)+\frac{1}{3}*Q(S_2,Left)+\frac{1}{3}*Q(S_2,Right);\\
          &=0+0.8*(\frac{1}{3}*0+\frac{1}{3}*0.267+\frac{1}{3}*0)\\
          &\approx 0.071\\
 \end{aligned} $
<br> 
&emsp;&emsp;$\begin{aligned}
Q(S_7,Left) &=r+\gamma\sum\limits_{a'}(\pi(a'|S_6)Q(S_6,a'))\\
          &=0+0.8*[\frac{1}{4}*Q(S_6,Up)+\frac{1}{4}*Q(S_6,Down)+\frac{1}{4}*Q(S_6,Left)+\frac{1}{4}*Q(S_6,Right)];\\
          &=0+0.8*(\frac{1}{4}*0+\frac{1}{4}*0.267+\frac{1}{4}*0.267+\frac{1}{4}*0)\\
          &\approx0.11\\
 \end{aligned} $
<br>
&emsp;&emsp;$\begin{aligned}
Q(S_7,Right) &=r+\gamma\sum\limits_{a'}(\pi(a'|S_8)Q(S_8,a'))\\
          &=0+0.8*[\frac{1}{4}*Q(S_8,Up)+\frac{1}{4}*Q(S_8,Down)+\frac{1}{4}*Q(S_8,Left)+\frac{1}{4}*Q(S_8,Right)];\\
          &=0+0.8*(\frac{1}{4}*0+\frac{1}{4}*0+\frac{1}{4}*0+\frac{1}{4}*0)\\
          &=0.00\\
 \end{aligned} $
<br> 
 &emsp;&emsp;按顺序计算完一轮后，得到动作值函数$Q(s,a)$，如表4.2（$\tau=1$）所示。<br>
 （3） 当$\tau=21$时， $|Q_\tau(s,a)-Q_{\tau-1}(s,a)|_∞<\theta$，认为$Q_\tau(s,a)$已经收敛于$q_\pi(s,a)$ ，计算得到的$q_\pi(s,a)$就是在策略$\pi$下的有效评估。<br>
&emsp;&emsp;随着迭代的进行，每轮动作值函数更新，如表4.1所示。表中动作按$Up$,$Down$,$Left$,$Right$顺序评估，不同动作值之间用“；”分割开。


In [6]:
import pandas as pd

def CaValue(Q):
    v = [0 for i in range(length)]
    for i in range(length):
        for a in action:
            newAction = getAction(a)
            v[i] += policy[i][newAction]*Q.loc[i,a]
    value = np.array(v).reshape(world_h, world_w)
    print(np.round(value, decimals=4))
    
def sumQ_nextstate(s,Q,visios):
    sum = 0
    for i in action:
        newAction = getAction(i)
        sum += policy[s][newAction]*Q.loc[s,i]
        
    return sum

def getReally(s):

    if s>=0 and s<=4: 
        return s+20;
    if s>=5 and s<=9: 
        return s+10
    if s>=15 and s<=19: 
        return s-10
    if s>=20 and s<=24: 
        return s-20
    else:
        return s
def policy_eval_Q(theta=0.000001):

    Q = pd.DataFrame(
        np.zeros((length, len(action))),  # q_table initial values
        columns=action,  # actions's name
    )
    iter = 0
    while True:
        k = -1
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in suqe:  # 遍历所有状态 [0~25]
            visio = False
            k += 1
            if s in [9, 20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            if s == 17:
                visio = True
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s,a)
                rewards = reward(next_state)
                if policy[s][newAction]  == 0:
                    continue
                if next_state == 12 :
                    q = rewards + gamma*(sumQ_nextstate(s,Q,visio))
                else:
                    q = rewards + gamma*(sumQ_nextstate(next_state,Q,visio))
                delta = max(delta, np.abs(q - Q.loc[s,a]))  # 更新差值

                Q.loc[s,a] = q  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
                
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(Q)
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
        # CaValue(Q)
    return Q
    

q = policy_eval_Q()
CaValue(q)

k= 1
       n          s          w         e
0    0.0   0.018963   0.000000  0.000000
1    0.0  -0.387674   0.007585  0.000000
2    0.0  -2.077535  -0.101357  0.000000
3    0.0  -0.234184  -0.581038  0.000000
4    0.0   3.000000  -0.217392  0.000000
5    0.0   0.071111   0.000000  0.000000
6    0.0  -1.957333   0.018963  0.000000
7    0.0 -10.000000  -0.387674  0.000000
8    0.0  -2.093383  -2.077535  3.000000
9    0.0   0.000000   0.000000  0.000000
10   0.0   0.266667   0.000000  0.000000
11   0.0   0.106667   0.071111 -9.964444
12   0.0   0.000000   0.000000  0.000000
13   0.0  -0.389096 -10.077819  0.000000
14   3.0  -0.101736  -2.093383  0.000000
15   0.0   1.000000   0.000000  0.000000
16   0.0   0.266667   0.266667  0.000000
17 -10.0   0.071111   0.106667  0.000000
18   0.0   0.018963  -1.964444  0.000000
19   0.0   0.007585  -0.389096  0.000000
20   0.0   0.000000   0.000000  0.000000
21   0.0   0.000000   1.000000  0.000000
22   0.0   0.000000   0.266667  0.000000
23   0.0   

k= 9
            n          s          w          e
0    0.000000  -1.106847   0.000000  -1.060652
1    0.000000  -1.877494  -0.867000  -1.274139
2    0.000000  -3.480470  -1.071635  -0.253479
3    0.000000  -0.782498  -1.281489   1.098608
4    0.000000   3.000000  -0.257434   0.000000
5   -0.852214  -1.434980   0.000000  -1.863482
6   -1.060652  -3.749413  -1.106847  -3.470556
7   -1.274139 -13.473034  -1.877494  -0.777683
8   -0.253479  -3.178542  -3.480470   3.000000
9    0.000000   0.000000   0.000000   0.000000
10  -1.088418  -0.559705   0.000000  -3.733053
11  -1.863482  -1.702995  -1.434980 -13.745607
12   0.000000   0.000000   0.000000   0.000000
13  -0.777683  -1.710563 -13.175124  -0.229339
14   3.000000  -0.697457  -3.178542   0.000000
15  -1.414539   1.000000   0.000000  -1.684354
16  -3.733053  -0.547310  -0.559705  -3.674906
17 -13.674906  -1.388186  -1.702995  -1.698278
18  -3.171042  -0.999925  -3.692873  -0.688977
19  -0.229339  -0.675561  -1.710563   0.000000
20   0.0

k= 21
            n          s          w          e
0    0.000000  -1.133476   0.000000  -1.087527
1    0.000000  -1.897768  -0.888401  -1.292133
2    0.000000  -3.494839  -1.087547  -0.263164
3    0.000000  -0.789479  -1.292147   1.094734
4    0.000000   3.000000  -0.263171   0.000000
5   -0.888374  -1.464417   0.000000  -1.897742
6   -1.087527  -3.773017  -1.133476  -3.494821
7   -1.292133 -13.494826  -1.897768  -0.789470
8   -0.263164  -3.189393  -3.494839   3.000000
9    0.000000   0.000000   0.000000   0.000000
10  -1.133442  -0.585137   0.000000  -3.772987
11  -1.897742  -1.729916  -1.464417 -13.773010
12   0.000000   0.000000   0.000000   0.000000
13  -0.789470  -1.728349 -13.189387  -0.239759
14   3.000000  -0.709732  -3.189393   0.000000
15  -1.464380   1.000000   0.000000  -1.729882
16  -3.772987  -0.572594  -0.585137  -3.718865
17 -13.718865  -1.417382  -1.729916  -1.728327
18  -3.189379  -1.023753  -3.718898  -0.709717
19  -0.239759  -0.693388  -1.728349   0.000000
20   0.

In [7]:
print(getReally(12))

12


<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.1 面向确定环境扫地机器人任务的动作值函数$q_\pi$评估过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_0$</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">×;0.00;0.00;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">0.00;×;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.27;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.07;-0.00</p></td>
        <td><p style="text-align:center">-10.00;0.07;0.11;0.00</p></td>
        <td><p style="text-align:center">×;3.00;-0.22;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_2$</p></td>
        <td><p style="text-align:center">0.11;×;1.00;0.07</p></td>
        <td><p style="text-align:center">-1.96;×;0.31;0.01</p></td>
        <td><p style="text-align:center">-0.39;×;-0.44;0.01</p></td>
        <td><p style="text-align:center">-11.96;-0.44;-0.66;-0.39</p></td>
        <td><p style="text-align:center">×;3.00;-0.09;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_{29}$</p></td>
        <td><p style="text-align:center">-1.73;×;1.00;-1.42</p></td>
        <td><p style="text-align:center">-3.72;×;-0.57;-1.02</p></td>
        <td><p style="text-align:center">-1.73;×;-1.42;-0.69</p></td>
        <td><p style="text-align:center">-13.72;-1.42;-1.73;-1.73</p></td>
        <td><p style="text-align:center">×;3.00;-0.26;×</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_{30}$</p></td>
        <td><p style="text-align:center">-1.73;×;1.00;-1.42</p></td>
        <td><p style="text-align:center">-3.72;×;-0.57;-1.02</p></td>
        <td><p style="text-align:center">-1.73;×;-1.42;-0.69</p></td>
        <td><p style="text-align:center">-13.72;-1.42;-1.73;-1.73</p></td>
        <td><p style="text-align:center">×;3.00;-0.26;×</p></td>
    </tr>
    
</table>


&emsp;&emsp;<b>例4.4</b> 利用算法4.2，使用$Q$值函数对随机环境扫地机器人任务进行策略评估。在其他条件与例4.2不变的情况下，采用异步计算方式，给出在策略$\pi$下，面向随机环境扫地机器人任务的动作值函数$q_\pi$评估过程。<br>
&emsp;&emsp;根据算法4.2，在随机策略$\pi(a|s)$下，利用$Q$值函数，随机环境扫地机器人任务的评估过程如下：<br>
&emsp;&emsp;（1）当$\tau$=0时（$\tau$是迭代的次数），对于所有的$(s,a)$初始化为：$Q(s,a)=0$ ；<br>
&emsp;&emsp;（2）当$\tau$=1时，以状态$S_7$的4个动作{$Up$,$Down$,$Left$,$Right$}为例。根据扫地机器人的随机环境MDP模型，$(S_7,Up)$以0.8的概率撞到障碍物，保持原地$S_7$($Q(S_7,Up)$=0.00，$Q(S_7,Down)$=0.00，$Q(S_7,Left)$=0.00，$Q(S_7,Right$)=0.00)不动，得到-10的惩罚；以0.15保持原地$S_7$($Q(S_7,Up)$=0.00，$Q(S_7,Down)$=0.00，$Q(S_7,Left)$=0.00，$Q(S_7,Right$)=0.00)不动，获得0的奖赏；以0.05滑到状态$S_2$($Q(S_2,Up)$=0.00，$Q(S_2,Left)$=0.19，$Q(S_7,Right)$=0.02)，获得0的奖赏。根据算法4.2计算得：<br>

&emsp;&emsp;$\begin{aligned}
Q(S_7,Up) &=p(s',r|s,Up)[r+\gamma\sum\limits_{a'}(\pi(a'|s')Q(s',a'))]\\
          &=p(S_7,r|S_7,Up)[r_1+\gamma\sum\limits_{a'}(\pi(a'|S_7)Q(S_7,a'))]\\
          &+p(S_7,r_2|S_7,Up)[r_2+\gamma\sum\limits_{a'}(\pi(a'|S_7)Q(S_7,a'))]\\
          &+p(S_2,r_3|S_7,Up)[r_3+\gamma\sum\limits_{a'}(\pi(a'|S_2)Q(S_2,a'))]\\
          &=0.8*[-10+0.8*(\frac{1}{4}*Q(S_7,Up)+\frac{1}{4}*Q(S_7,Down)+\frac{1}{4}*Q(S_7,Left)+\frac{1}{4}*Q(S_7,Right))]\\
          &=0.15*[0+0.8*(\frac{1}{4}*Q(S_7,Up)+\frac{1}{4}*Q(S_7,Down)+\frac{1}{4}*Q(S_7,Left)+\frac{1}{4}*Q(S_7,Right))]\\
          &=0.05*[0+0.8*(\frac{1}{3}*Q(S_2,Up)+\frac{1}{3}*Q(S_2,Left)+\frac{1}{3}*Q(S_2,Right))]\\
          &=0.8*[-10+0.8*(\frac{1}{4}*0.00+\frac{1}{4}*0.00+\frac{1}{4}*0.00+\frac{1}{4}*0.00)]\\
          &=0.15*[0+0.8*(\frac{1}{4}*0.00+\frac{1}{4}*0.00+\frac{1}{4}*0.00+\frac{1}{4}*0.00)]\\
          &=0.05*[0+0.8*(\frac{1}{3}*0.00+\frac{1}{3}*0.19+\frac{1}{3}*0.02)]\\
          &=-8.00\\
          ;
 \end{aligned} $
<br>
同理可以计算动作值函数$Q(S_7,Down)$=-0.70，$Q(S_7,Left)$=-0.19，$Q(S_7,Right)$=-0.26。<br>  
&emsp;&emsp;按顺序计算完一轮后，得到动作值函数$Q(s,a)$，如表4.2（$\tau$=1）所示。<br>
&emsp;&emsp;（3） 当$\tau$=34时，$|Q_\tau(s,a)-Q_{\tau-1}(s,a)|_∞<\theta$ ，认为$Q_\tau(s,a)$已经收敛于$q_\pi(s,a)$，计算得到的$q_\pi(s,a)$就是在策略$\pi$下的有效评估。<br>
&emsp;&emsp;随着迭代的进行，每轮动作值函数更新，如表4.2所示。<br>



In [8]:

    
def policy_eval_Q_random(theta=0.0000002):

    Q = pd.DataFrame(
        np.zeros((length, len(action))),  # q_table initial values
        columns=action,  # actions's name
    )
    iter = 0
    while True:
        k = -1
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in suqe:  # 遍历所有状态 [0~25]
            visio = False
            k += 1
            if s in [9, 20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            if s == 17:
                visio = True
            # [[-0.7954 - 1.0218 - 1.2655 - 0.1564  1.369]
            #  [-1.066 - 1.9614 - 3.8893 - 0.7455  0.]
            # [-1.4346 - 4.176
            # 0. - 3.5631 - 0.0563]
            # [-0.489 - 1.7904 - 4.1252 - 1.7891 - 0.6118]
            # [0. - 0.4778 - 1.3917 - 0.9611 - 0.5992]]
            for a in action:
                newAction = getAction(a)
                env_action=envActionPolicy(a)
                next_state = next_states(s,a)
                env_state = next_states(s,env_action)
                rewards = reward(next_state)
                env_rewards = reward(env_state)
                if policy[s][newAction]  == 0:
                    continue
                if next_state == 12 :
                    q = 0.8*(rewards+gamma*sumQ_nextstate(s,Q,visio))+0.15*(gamma*sumQ_nextstate(s,Q,visio))+0.05*(env_rewards + gamma*(sumQ_nextstate(env_state,Q,visio)))
                    if visio == True:
                        print("q=%.2f=0.8*(%.2f+%.2f*%.2f)+0.15*(%.2f*%.2f)+0.05*(%.2f+%.2f*%.2f)"
                              %(q,rewards,gamma,sumQ_nextstate(s,Q,visio),gamma,sumQ_nextstate(s,Q,visio),env_rewards,gamma,sumQ_nextstate(env_state,Q,visio)))
                else:
                    q = 0.8*(rewards+gamma*sumQ_nextstate(next_state,Q,visio))+0.05*(env_rewards + gamma*sumQ_nextstate(env_state,Q,visio))\
                        +0.15*gamma*sumQ_nextstate(s,Q,visio)
                    if visio == True:
                        print("q=%.2f=0.8*(%.2f+%.2f*%.2f)+0.15*(%.2f*%.2f)+0.05*(%.2f+%.2f*%.2f)" % (q,
                        rewards, gamma, sumQ_nextstate(next_state, Q,visio), gamma, sumQ_nextstate(s, Q,visio), env_rewards, gamma,
                        sumQ_nextstate(env_state, Q,visio)))
                delta = max(delta, np.abs(q - Q.loc[s,a]))  # 更新差值
                Q.loc[s,a] = q  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(Q)
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
        # CaValue(Q)
    return Q
initPolicy()    
q=policy_eval_Q_random()
CaValue(q)

q=-8.00=0.8*(-10.00+0.80*0.00)+0.15*(0.80*0.00)+0.05*(0.00+0.80*0.07)
q=-0.70=0.8*(0.00+0.80*0.07)+0.15*(0.80*-2.00)+0.05*(-10.00+0.80*0.00)
q=-0.19=0.8*(0.00+0.80*0.11)+0.15*(0.80*-2.17)+0.05*(0.00+0.80*0.00)
q=-0.26=0.8*(0.00+0.80*0.00)+0.15*(0.80*-2.22)+0.05*(0.00+0.80*0.11)
k= 1
           n         s         w         e
0   0.000000  0.010965  0.000000  0.000877
1   0.000000 -0.242339 -0.005904 -0.009693
2   0.000000 -1.501311 -0.115079 -0.068095
3   0.000000 -0.115938 -0.363994 -0.041657
4   0.000000  2.400000  0.080728  0.000000
5   0.002864  0.045933  0.000000  0.002602
6  -0.084496 -1.354471 -0.032204 -0.043450
7  -0.500000 -8.095000 -0.500189 -0.288002
8  -0.090714 -1.454138 -1.397656  2.217893
9   0.000000  0.000000  0.000000  0.000000
10  0.011966  0.191933  0.000000  0.010875
11  0.004216  0.067588 -0.452028 -8.069379
12  0.000000  0.000000  0.000000  0.000000
13 -0.015874 -0.254463 -8.051364 -0.749651
14  2.396663  0.192480 -1.313329  0.000000
15  0.050000  0.802000  0.00

24  -0.481892   0.000000  -0.705082   0.000000
q=-11.18=0.8*(-10.00+0.80*-4.12)+0.15*(0.80*-4.12)+0.05*(0.00+0.80*-1.39)
q=-1.88=0.8*(0.00+0.80*-1.39)+0.15*(0.80*-4.12)+0.05*(-10.00+0.80*0.00)
q=-1.71=0.8*(0.00+0.80*-1.79)+0.15*(0.80*-4.12)+0.05*(0.00+0.80*-1.78)
q=-1.71=0.8*(0.00+0.80*-1.78)+0.15*(0.80*-4.12)+0.05*(0.00+0.80*-1.79)
k= 13
            n          s          w          e
0    0.000000  -0.805229   0.000000  -0.777081
1    0.000000  -1.415546  -0.678912  -0.961582
2    0.000000  -2.689414  -0.809649  -0.291244
3    0.000000  -0.500986  -0.772385   0.807504
4    0.000000   2.619164   0.119773   0.000000
5   -0.688771  -1.073401   0.000000  -1.420953
6   -1.052201  -2.945284  -1.069242  -2.763710
7   -1.773967 -11.003427  -1.748914  -1.020399
8   -0.330569  -2.374534  -2.426731   2.155240
9    0.000000   0.000000   0.000000   0.000000
10  -0.867560  -0.523719   0.000000  -2.896715
11  -1.823065  -1.721413  -1.915230 -11.227224
12   0.000000   0.000000   0.000000   0.000000
1

k= 20
            n          s          w          e
0    0.000000  -0.809380   0.000000  -0.781080
1    0.000000  -1.418677  -0.682163  -0.964277
2    0.000000  -2.691572  -0.811999  -0.292807
3    0.000000  -0.502092  -0.773901   0.806774
4    0.000000   2.619040   0.118940   0.000000
5   -0.694161  -1.077754   0.000000  -1.425698
6   -1.056226  -2.948736  -1.073072  -2.767056
7   -1.776567 -11.006398  -1.751735  -1.022210
8   -0.332055  -2.376064  -2.428555   2.154980
9    0.000000   0.000000   0.000000   0.000000
10  -0.873767  -0.527624   0.000000  -2.901976
11  -1.827861  -1.725288  -1.919145 -11.230986
12   0.000000   0.000000   0.000000   0.000000
13  -0.976176  -1.602336 -10.710164  -0.963576
14   2.368778  -0.248283  -2.289365   0.000000
15  -0.926633   0.683969   0.000000  -1.223897
16  -2.906391  -0.687537  -0.692685  -2.874321
17 -11.190595  -1.885601  -1.712302  -1.711519
18  -2.533443  -0.972209  -2.879146  -0.771176
19  -0.133380  -0.459085  -1.242836   0.000000
20   0.

k= 28
            n          s          w          e
0    0.000000  -0.809520   0.000000  -0.781215
1    0.000000  -1.418783  -0.682273  -0.964369
2    0.000000  -2.691645  -0.812078  -0.292860
3    0.000000  -0.502130  -0.773952   0.806749
4    0.000000   2.619036   0.118912   0.000000
5   -0.694343  -1.077900   0.000000  -1.425858
6   -1.056362  -2.948852  -1.073201  -2.767169
7   -1.776655 -11.006499  -1.751830  -1.022272
8   -0.332105  -2.376116  -2.428617   2.154971
9    0.000000   0.000000   0.000000   0.000000
10  -0.873975  -0.527754   0.000000  -2.902151
11  -1.828022  -1.725418  -1.919276 -11.231112
12   0.000000   0.000000   0.000000   0.000000
13  -0.976237  -1.602419 -10.710230  -0.963626
14   2.368767  -0.248340  -2.289415   0.000000
15  -0.926839   0.683936   0.000000  -1.224085
16  -2.906569  -0.687664  -0.692810  -2.874502
17 -11.190783  -1.885735  -1.712429  -1.711657
18  -2.533538  -0.972325  -2.879262  -0.771278
19  -0.133441  -0.459175  -1.242920   0.000000
20   0.

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.2 面向随机环境扫地机器人任务的动作值函数$q_\pi$评估过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_0$</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">×;0.00;0.00;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">0.00;×;0.80;0.08</p></td>
        <td><p style="text-align:center">0.00;×;0.19;0.02</p></td>
        <td><p style="text-align:center">0.00;×;0.04;0.00</p></td>
        <td><p style="text-align:center">-8.00;-0.70;-0.19;-0.26</p></td>
        <td><p style="text-align:center">×;2.40;0.08;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_2$</p></td>
        <td><p style="text-align:center">0.11;×;0.84;0.14</p></td>
        <td><p style="text-align:center">-1.45;×;0.18;-0.02</p></td>
        <td><p style="text-align:center">-0.25;×;-0.28;-0.04</p></td>
        <td><p style="text-align:center">-9.76;-1.10;-0.77;-0.64</p></td>
        <td><p style="text-align:center">×;2.60;0.18;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_{33}$</p></td>
        <td><p style="text-align:center">-1.22;×;0.69;-0.90</p></td>
        <td><p style="text-align:center">-2.86;×;-0.51;-0.80</p></td>
        <td><p style="text-align:center">-1.30;×;-0.51;-0.80</p></td>
        <td><p style="text-align:center">-11.19;-1.89;-1.71;-1.71</p></td>
        <td><p style="text-align:center">×;2.62;0.19;×</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_{34}$</p></td>
        <td><p style="text-align:center">-1.22;×;0.69;-0.90</p></td>
        <td><p style="text-align:center">-2.86;×;-0.51;-0.80</p></td>
        <td><p style="text-align:center">-1.30;×;-0.51;-0.80</p></td>
        <td><p style="text-align:center">-11.19;-1.89;-1.71;-1.71</p></td>
        <td><p style="text-align:center">×;2.62;0.19;×</p></td>
    </tr>
</table>

&emsp;&emsp;策略的优劣性可以由值函数来评价。通过策略评估迭代得到值函数，再利用动作值函数来寻找更好的策略。假设已知某一策略$\pi$的值函数$v_\pi$或$q_\pi$ ，目的是寻找一个更优策略$\pi'$。下面从特殊情况到一般情况对策略改进方法进行说明，最后推导出最优策略的获取方法。<br>
&emsp;&emsp;（1）特殊情况<br>
&emsp;&emsp;针对单一状态$s$和特定动作$a$，制定如下约定以获得新策略$\pi'(\pi'\neq\pi)$：<br>
&emsp;&emsp;$\blacktriangleright$&emsp;在状态$s$下选择一个新动作$a(a\neq\pi(s))$ ；<br>
&emsp;&emsp;$\blacktriangleright$&emsp;保持后续（其他）状态所执行的动作与原策略$\pi$给出的动作相同。<br>
&emsp;&emsp;在这种情况下，根据动作值函数贝尔曼方程，得到的价值为：<br>
 $$\begin{aligned}
 q_\pi(s,a) &=\mathbb{E}_\pi(R_{t+1}+\gamma{v_\pi(S_{t+1})}|S_t=s,A_t=a)\\
          &=\sum_{s',r}p(s',r|s,a)[r+\gamma{v_\pi(s')}]\\
 \end{aligned} \tag{4.3}$$
&emsp;&emsp;若$q_\pi(s,a)\ge v_\pi(s)$成立，则说明满足以上约定的策略$\pi'$优于或等价于$\pi$ 。<br>
&emsp;&emsp;（2）一般情况<br>
&emsp;&emsp;将单一状态和特定动作的情况进行拓展。对任意状态$s\in{S_s}$，若存在任意的两个确定策略$\pi$和$\pi'（\pi'\neq\pi）$满足策略改进定理（policy improvement theorem），则说明在状态$s$处采取策略$\pi'$时，能得到更大的值函数，即$\pi'$优于或等价于$\pi$ 。策略改进定理为：<br>
$$\begin{aligned}
 q_\pi(s,\pi'(s))\ge{v_\pi(s)}
 \end{aligned} \tag{4.4}$$
&emsp;&emsp;此时，对任意状态$s$，执行策略$\pi'$时，必定能够得到更大的期望回报，即存在：<br>
$$\begin{aligned}
 v_{\pi'}(s)\ge{v_\pi(s)}
 \end{aligned} \tag{4.5}$$
&emsp;&emsp;对策略改进定理进行推导，利用式（4.3），基于式（4.2）等号左侧的$q_\pi$进行多次展开，并不断应用式（4.3）直到得到$v_{\pi'(s)}$ ：

 $$\begin{aligned}
 v_\pi&\le{q_\pi(s,\pi'(s))} \\
 &=\mathbb{E}[R_{t+1}+\gamma{v_\pi(S_{t+1})|S_t=s,A_t=\pi'(s)}] \quad\quad\quad由式(4，2)推出\\
 &=\mathbb{E}_{\pi'}[R_{t+1}+\gamma{v_\pi(S_{t+1})|S_t=s)}]\quad\quad\quad\quad\quad\quad将条件\pi'(s)提出\\
 &\le{\mathbb{E}_{\pi'}[R_{t+1}+\gamma{q_\pi(S_{t+1},\pi'(S_{t+1}))|S_t=s}]\quad\quad\quad由式(4，4)推出,v_\pi(S_{t+1}\le{q_\pi(S_\pi(S_{t+1},\pi'(S_{t+1})})}\\
 &=\mathbb{E}_{\pi'}[R_{t+1}+\gamma{\mathbb{E}_{\pi'}}[R_{t+2}+\gamma\pi(S_{t+2})]|S_t=s]\quad\quad由式(4,2)推出\\
 &\le{\mathbb{E}_{\pi'}[R_{t+1}+\gamma{R_{t+2}}+\gamma^2v_\pi(S_{t+2})|S_t=s]}\quad\quad\quad期望展开\\
 &\le{\mathbb{E}_{\pi'}[R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+\gamma^3v_\pi(S_{t+3})|S_t=s]}\\
 &...\\
 &\le{\mathbb{E}_{\pi'}[R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+...|S_t=s]=\mathbb{E}_{\pi'}[G_t|S_t=s]=v_{\pi'}(s)}
 \end{aligned} \tag{4.5}$$
 &emsp;&emsp;由上述推导可知，给定一个策略及其值函数估计值，就能判定改变某个状态的动作对策略的影响。于是对每个状态$s$都选择最优动作，即基于贪心策略选取最大动作值函数对应的动作，构建一个新的贪心策略$\pi$，满足如下等式：<br>
  $$\begin{aligned}
\pi'(s)&=\mathop{\arg\max}_{a}q_\pi(s,a)\\
&= \mathop{\arg\max}_{a}\mathbb{E}(R_{t+1}+\gamma{v_\pi(S_{t+1})|S_t=s,A_t=a})\\
&=\mathop{\arg\max}_{a}\sum_{s',r}p(s',r|s,a)[r+\gamma{v_\pi(s')}]
 \end{aligned} \tag{4.6}$$
 &emsp;&emsp;其中，$\mathop{\arg\max}\limits_{a}$表示能够使得表达式值最大化的$a$，如果结果有多个满足条件的值时，则随机取其一。 <br>
&emsp;&emsp;贪心策略选取了短期内最优的动作，即根据$v_\pi$向前单步搜索，式（4.6）构造出的策略满足策略改进定理：<br>
$$\begin{aligned}
v_{\pi'}(s)=q_\pi(s,\pi'(s))=\max_aq_\pi(s,a)\ge{q_\pi(s,\pi(s))=v_\pi(s)}
 \end{aligned} \tag{4.7}$$
&emsp;&emsp;这样一种贪心策略的概率分布呈$one-hot$分布，该策略也称为确定贪心策略。其满足如下关系：<br>
$$
\pi'(a|s)=\begin{cases}
1 & 
a=\mathop{\arg\max}\limits_{a}q_\pi(s,a)\\
0 & 其他\\
 \end{cases} \tag{4.8}$$
&emsp;&emsp;另一种贪心策略称为随机贪心策略，其概率分布为：对于所有的贪心动作$a'$，赋予一个$(0,1]$的值，且其满足如下关系：
$$
\pi'(a|s)=\begin{cases}
\pi'(a'|s) & \pi'(a'|s)\in(0,1]且
\sum\limits_{a'}\pi'(a'|s)=1\\
0 & 其他\\
 \end{cases} \tag{4.9}$$
&emsp;&emsp;（3）策略迭代中的最优策略<br>
&emsp;&emsp;通过策略改进，可以得到更优策略$\pi'$，而强化学习的目标是获得最优策略$\pi_*$。假设贪心策略$\pi'$与原策略$\pi$一样好，根据式（4.6）可得，对任意$s\in{S_s}$，存在：<br>
 $$\begin{aligned}
v_{\pi'}(s)=\max_a\sum_{s',r}p(s',r|s,a)[r+\gamma{v_{\pi'}(s')}]
 \end{aligned} $$
 &emsp;&emsp;该式与贝尔曼最优方程（3.24）的形式完全相同，因此$v_{\pi'}$一定等于$v_*$，且$\pi$和$\pi'$均为最优策略。综上所述，策略改进可以得到最优策略。<br>

### 4.1.3 策略迭代
&emsp;&emsp;策略迭代的关键部分是策略评估，首先评估状态的价值，然后根据状态的动作值进行相应的策略改进，并进行下一轮评估和改进，直到策略稳定。策略改进可以通过求解静态最优化问题来实现，通过状态动作值来选择动作，通常比策略评估容易。<br>
#### 4.1.3.1 基于状态值函数的策略迭代
&emsp;&emsp;基于状态值函数的策略迭代算法主要包括以下3个阶段：<br>
&emsp;&emsp;（1）初始化策略函数$\pi(a|s)$和状态值函数$V_0(s)$；<br>
&emsp;&emsp;（2）策略评估：在当前策略$\pi$下，使用贝尔曼方程更新状态值函数$V_\tau(s)$，直到收敛于$v_\pi(s)$，再根据式（4.3）计算出$q_\pi(s,a)$。<br>
&emsp;&emsp;（3）策略改进：基于$q_\pi(s,a)$，通过贪心策略得到更优策略$\pi'$，满足式（4.9）。<br>
&emsp;&emsp;直到策略不再发生变化，迭代产生的状态值函数渐近地收敛到$v_*(s)$，同时得到最优策略$\pi_*(s)$ 。<br>
&emsp;&emsp;通常基于状态值函数的策略改进是通过状态值函数$v_\pi(s)$进行的，但需要经过一步搜索，得到动作值函数$q_\pi(s,a)$，而后进行改进策略。基于状态值函数估算最优策略的策略迭代算法，如算法4.3所示。<br>


<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.3</b> 基于随机MDP的状态值函数$v_\pi$策略迭代算法</font>
<br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
&emsp;<font size=3.5>初始策略$\pi(a|s)$，动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
<font size=3.5>&emsp;1.&emsp;对任意$s\in{S_s}$，初始化状态值函数：如$V(s)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实值</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>策略评估：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略评估$l=1,2,\cdots$ </font><br>
<font size=3.5><b>策</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;在当前策略$\pi(a|s)$下，通过算法4.1策略评估迭代，得到值函数$v_\pi{:}V(s)$ </font><br>

&emsp;&emsp;<font size=3.5><b>策略改进：</b></font>
<font size=3.5>5.&emsp;&emsp;&emsp;$policy$_$stable$←<b>True</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;<b>for</b>&emsp;每个状态$s$&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;
$old$_$policy←\pi(a|s)$
</font><br>
<font size=3.5><b>略</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;
$Q(s,a)=\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{V(s')}]$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;&emsp;&emsp;
$max$_$action←\mathop{\arg\max}\limits_{a}Q(s,a)$;&emsp;$max$_$conut←count(max(Q(s,a)))$
</font><br>
<font size=3.5><b>迭</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>for</b>&emsp;每个状态-动作对$(s,a')$&emsp;<b>do</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>if</b>&emsp;$a' == max$_$action$&emsp;<b>then</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=\frac{1}{(max\_count)}$
</font><br>
<font size=3.5><b>代</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>else</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=0$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end</b>&emsp;<b>if</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>16.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>17.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>if</b>&emsp;$old$_$policy\neq\pi(a|s)$&emsp;<b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>18.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$policy$-$stable$←<b>False</b></font><br>
</font>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>19.&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>20.&emsp;
<b>until</b>&emsp;$policy$_$stable$=<b>True</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;<font size=3.5>$v_*=V,\pi_*=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>
&emsp;&emsp;<b>例4.5</b> 将基于状态值函数的策略迭代算法4.3应用于例4.1的确定环境扫地机器人任务中，经过多轮迭代后，得到表4.3所示的值函数和策略迭代更新过程。<br>
 图4.5给出了面向确定环境扫地机器人任务的状态值函数及策略迭代更新过程图。

In [9]:
def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    while True:
        delta = 0  # 定义最大差值，判断是否有进行更新

        for s in [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]:  # 遍历所有状态 [0~25]
            if s in [9, 20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s,a)
                rewards = reward(next_state)
                if next_state == 12:
                    v += policy[s][newAction] * (rewards + gamma * V[s])
                else:
                    v += policy[s][newAction] * (rewards + gamma * V[next_state])

            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            V[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)

        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
    return V  # 一轮迭代结束后，状态值函数暂时固定




def Caculate_Q(s, V, num, discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    Q[:][:] = -100
    for a in action:  # 遍历能走的动作
        # for prob, next_state, reward, done in env.P[s][a]:
        #     Q[s][a] += prob * (reward + discount_factor * V[next_state])  # 计算当前状态s下的动作值函数列表 [q1,q2,q3,q4]
        next_state = next_states(s,a)
        if next_state == s: #碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        if s == 12:
            Q[s][numberA] = rewards + discount_factor * V[s]
        else:
            Q[s][numberA]= rewards + discount_factor * V[next_state]
        print("Q[%s][%d]  %f = %s + 0.8 * %.2f:"%(num,numberA,Q[s][numberA],rewards,V[next_state]))
    return Q



def policy_improvement(V, policy):  # 策略改进
    """
    Returns:
        policy: the optimal policy, a matrix of shape [S, A] where each state s contains a valid probability distribution over actions.
        V: the value function for the optimal policy.
    """
    k = -1
    policy_stable = True  # Will be set to false if we make any changes to the policy
    for s in [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]:
        k +=1
        if np.all(policy[s]==0):
            continue
        old_a = np.argmax(policy[s])  # 记录当前策略在该状态s下所选择的动作 —— 选择概率最高的动作
        Q = Caculate_Q(s, V, k)  # 在当前状态和策略下，计算动作值函数 —— 要判断在状态s下选择其他动作的好坏，就需要获得状态s的动作值函数
        best_a = np.argmax(Q[s])  # 采用贪婪策略获得状态s的最优动作；如果往两个方向前进都可以得到最优解，会随机选其一
        if old_a != best_a:  # 判定策略是否稳定
            policy_stable = False  # 动作还在变化，不稳定状态
        policy[s] = np.eye(len(action))[best_a]  # 基于贪婪法则更新策略，形如[0，0，1，0]； -》np.eye(*)：构建对角单位阵
        # 经过一次策略改进后的策略将不再拥有多个动作可供备选，取而代之的是在某种状态下的确定策略
    return V, policy_stable


def Policy_Iterration():
    initPolicy()
    k = 1
    while True:  # Evaluate the current policy
        print("迭代次数",k)
        V = policy_eval()  # 得到当前策略下的收敛状态值函数 —— 与Value_Iteration的不同点，多了policy_eval()函数。policy会在迭代中改变
        V, policy_stable = policy_improvement(V, policy)
        v = np.array(V).reshape(world_h, world_w)
        policy1 = np.array(policy).reshape(length,len(action))
        print(np.round(v, decimals=4))
        # print(np.round(policy1, decimals=2))
        # print(np.argmax(policy, axis=1).reshape(world_h, world_w))  # 输出在每个状态上会采取的动作 0向上，1向右，2向下，3向右
        print("*" * 100)
        k+=1
        if policy_stable:  # # If the policy is stable we've found an optimal policy. Return it
            return policy, V
policy, V = Policy_Iterration()

迭代次数 1
Q[0][0]  -0.585183 = 0 + 0.8 * -0.73:
Q[0][3]  -0.572640 = 0 + 0.8 * -0.72:
Q[1][0]  -1.729965 = 0 + 0.8 * -2.16:
Q[1][2]  1.000000 = 1 + 0.8 * 0.00:
Q[1][3]  -1.417435 = 0 + 0.8 * -1.77:
Q[2][0]  -3.718945 = 0 + 0.8 * -4.65:
Q[2][2]  -0.572640 = 0 + 0.8 * -0.72:
Q[2][3]  -1.023796 = 0 + 0.8 * -1.28:
Q[3][0]  -1.728382 = 0 + 0.8 * -2.16:
Q[3][2]  -1.417435 = 0 + 0.8 * -1.77:
Q[3][3]  -0.693420 = 0 + 0.8 * -0.87:
Q[4][0]  -0.709755 = 0 + 0.8 * -0.89:
Q[4][2]  -1.023796 = 0 + 0.8 * -1.28:
Q[5][0]  -1.464471 = 0 + 0.8 * -1.83:
Q[5][1]  1.000000 = 1 + 0.8 * 0.00:
Q[5][3]  -1.729965 = 0 + 0.8 * -2.16:
Q[6][0]  -3.773060 = 0 + 0.8 * -4.72:
Q[6][1]  -0.572640 = 0 + 0.8 * -0.72:
Q[6][2]  -0.585183 = 0 + 0.8 * -0.73:
Q[6][3]  -3.718945 = 0 + 0.8 * -4.65:
Q[7][0]  -10.000000 = -10 + 0.8 * 0.00:
Q[7][1]  -1.417435 = 0 + 0.8 * -1.77:
Q[7][2]  -1.729965 = 0 + 0.8 * -2.16:
Q[7][3]  -1.728382 = 0 + 0.8 * -2.16:
Q[8][0]  -3.189413 = 0 + 0.8 * -3.99:
Q[8][1]  -1.023796 = 0 + 0.8 * -1.28:
Q[8][2]

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.3 面向确定环境扫地机器人任务的状态值函数策略迭代更新过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    
  <tr>
        <td><p style="text-align:center">$v_0$</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$\pi_0$</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.25;0.25;0.25;0.25</p></td>
        <td><p style="text-align:center">0.00;0.5;0.5;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$v_1$</p></td>
        <td><p style="text-align:center">-0.72</p></td>
        <td><p style="text-align:center">-1.77</p></td>
        <td><p style="text-align:center">-1.28</p></td>
        <td><p style="text-align:center">-4.65</p></td>
        <td><p style="text-align:center">1.37</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">-1.73;×;1.00;-1.42</p></td>
        <td><p style="text-align:center">-3.72;×;-0.57;-1.02</p></td>
        <td><p style="text-align:center">-1.73;×;-1.42;-0.69</p></td>
        <td><p style="text-align:center">-10.00;-1.42;-1.73;-1.73</p></td>
        <td><p style="text-align:center">×;3.00;-0.26;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_1$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$v_2$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">0.80</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_2$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_2$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
       <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_4$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
   <tr>
        <td><p style="text-align:center">$v_5$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">1.23</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_5$</p></td>
        <td><p style="text-align:center">0.98;×;1.00;0.98</p></td>
        <td><p style="text-align:center">1.23;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.98;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.98;0.98;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_5$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$v_6$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">1.23</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_6$</p></td>
        <td><p style="text-align:center">0.98;×;1.00;0.98</p></td>
        <td><p style="text-align:center">1.23;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.98;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.98;0.98;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_6$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_*$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
</table>
<center> 图4.5给出了面向确定环境扫地机器人任务的状态值函数及策略迭代更新过程图。</center><br>
<center><img src="./image/图4.6.jpg" ></center><br>
<center>图4.5&ensp;面向确定环境扫地机器人任务的状态值函数及策略迭代更新图</center>
&emsp;&emsp;在图4.5中，左边一列给出了每一轮（$l$=0,1,2,3,4,5）状态值函数的迭代更新过程。当$l$=4和5时，策略不再发生变化（这里状态值也不再变化），策略评估迭代收敛。右侧一列给出了相应的策略改进过程。依据对动作值函数的贪心，获得较好策略，对策略进行改进。从图4.5可以看出，每个状态的最优策略始终选择动作值最大的动作。<br>
&emsp;&emsp;<b>例4.6</b> 汽车租赁问题。杰克经营两家异地的汽车租赁场（A，B租赁场），每天都会有客户来租车。如果每个租赁场有可供外租的汽车，每租出一辆汽车，杰克都会获得10美元的租金。为了保证每个租赁场有足够的车辆可租，每天晚上杰克会在两个租赁场之间移动车辆，每辆车的移车费用为2美元。假设每个租赁场的租车和还车的数量是一个泊松随机量，即期望数量n的概率为$\frac{\lambda^n}{n!}e^{-\lambda}$ ，其中$\lambda$为期望值。假设在两个租赁场租车的$\lambda$分别是$\lambda_{A1}$=3,$\lambda_{B1}$=4， ；还车的$\lambda$分别为$\lambda_{A2}$=3,$\lambda_{B2}$=2。<br>
&emsp;&emsp;为了简化问题，给定3个假设：（1）任何一个租赁场车辆总数不超过20辆车；（2）当天还回的车辆第2天才能出租。（3）两个租车场之间每天最多可移车数量为5辆。<br>
&emsp;&emsp;利用策略迭代方法，在折扣率$\gamma$=0.9时，计算两个租赁场点之间的最优移车策略。<br>
&emsp;&emsp;将汽车租赁问题描述为一个持续的有限MDP模型。其中时刻$t$按天计算；状态为每天租还车结束时，两个租赁场的车辆数；动作为每天晚上在两个租赁场之间移动的车辆数目。<br>
&emsp;&emsp;该问题的MDP模型为：<br>
&emsp;&emsp;（1）状态空间：两个租赁场每天租还结束时，可供出租的车辆数组成的二维向量，状态共$21\times21$=441个，即$S={[0,0],[0,1],[0,2],\dots,[10,10],[10,11],\dots,[20,19],[20,20]}$。用$s$表示状态空间中的某一个状态$[s_A,s_B]$。例如,$[3,5]$表示每天租还结束后，A租赁场可供出租3辆车、B租赁场可供出租5辆车的状态。<br>
&emsp;&emsp;（2）动作空间：每天晚上在两个租赁场之间移动车辆的数目。根据任务的假设，可移动车辆数目不超过5辆。设A租赁场向B租赁场移车为“-”，B租赁场向A租赁场移车为“+”。该问题的动作空间中共包含11个动作，即：$A(s)={-5,-4,-3,-2,-1,0,+1,+2,+3,+4,+5}$。用$a$表示动作空间中的某一个动作。例如，-2表示A租赁场向B租赁场移车2辆；+3表示B租赁场向A租赁场移车3辆。<br>
&emsp;&emsp;（3）状态转移函数：<br>
&emsp;&emsp;令$s=[s_A,s_B],s'=[s_A',s_B']$，那么状态转移函数为：$p([s_A,s_B],a,[s_A',s_B'])$，即在当前状态$s=[s_A,s_B]$下，采取动作$a$，到达下一状态$s'=[s_A',s_B']$的概率。设$n_A$为A租赁场当天租掉的车辆数；$n_B$为B租赁场当天租掉的车辆数。$m_A$为A租赁场当天还回的车辆数；$m_B$为B租赁场当天还回的车辆数。假设当前状态为$s=[2,5]$，采取+1动作后，中间状态为$s_L=[3,4]$，当下一状态为$s'=[1,2]$，对于A租赁场来说，一天的租掉、还回车辆$(n_A,m_A)$可能为：(2,0)、(3,1)共2种情况；对于B租赁场来说，一天的租掉、还回车辆$(n_B,m_B)$可能为：(2,0) 、(3,1) 、(4,2)共3种情况。那么在该情况下，状态转移函数$p([2,5],+1,[1,2])$计算为：<br>
$$\begin{aligned}
p([2,5],+1,[1,2])&=(\frac{\lambda_{A1}^2}{2!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^0}{0!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^2}{2!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^2}{0!}e^{-\lambda_{B1}})\\
&+(\frac{\lambda_{A1}^2}{2!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^0}{0!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^3}{3!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^1}{1!}e^{-\lambda_{B1}})\\
&+(\frac{\lambda_{A1}^2}{2!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^0}{0!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^4}{4!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^2}{2!}e^{-\lambda_{B1}})\\
&+(\frac{\lambda_{A1}^3}{3!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^1}{1!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^2}{2!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^0}{0!}e^{-\lambda_{B1}})\\
&+(\frac{\lambda_{A1}^3}{3!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^1}{1!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^3}{3!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^1}{1!}e^{-\lambda_{B1}})\\
&+(\frac{\lambda_{A1}^3}{3!}e^{-\lambda_{A1}})*(\frac{\lambda_{A1}^1}{1!}e^{-\lambda_{A1}})*(\frac{\lambda_{B1}^4}{4!}e^{-\lambda_{B1}})*(\frac{\lambda_{B1}^2}{2!}e^{-\lambda_{B1}})\\
\end{aligned} $$
<br>
&emsp;&emsp;根据任务可知，这里$\lambda_{A1}$=3，$\lambda_{B1}$=4，$\lambda_{A2}$=3，$\lambda_{B2}$=2。<br>
&emsp;&emsp;综上所述，如果$0\le{S_A'-S_A-a+n_A\le20}$且$0\le{S_B'}-S_B+a+n_B\le20$，则有：<br>
$$\begin{aligned}
p(s,a,s')&=p([s_A,s_B],a,[s_A',s_B'])\\
&=\sum\limits_{n_A=0}^{S_A+a}\sum\limits_{n_B=0}^{S_B-a}(\frac{\lambda_{A1}^{n_A}}{n_A!}e^{-\lambda_{A1}})*(\frac{\lambda_{A2}^{(S_A'-S_A-a+n_A)}}{(S_A'-S_A-a+n_A)!}e^{-\lambda_{A2}})\\
&*(\frac{\lambda_{B1}^{n_B}}{n_B!}e^{-\lambda_{B1}})*(\frac{\lambda_{B2}^{(S_B'-S_B-a+n_B)}}{(S_B'-S_B-a+n_B)!}e^{-\lambda_{B2}})
\end{aligned} $$
<br>
&emsp;&emsp;（4）立即奖赏：该问题的立即奖赏函数为：$r([s_A,s_B],a,[s'_A,s'_B])$，即在当前状态$s=[s_A,s_B]$下，采取动作$a$，到达下一状态$s'=[s'_A,s'_B]$得到的立即奖赏。<br>
&emsp;&emsp;如果$0\le{S'_A-S_A-a+n_A\le20}$且$0\le{S'_B-S_B-a+n_B\le20}$，则立即奖赏由3部分组成，分别是：<br>
&emsp;&emsp;$\blacktriangleright$两个租赁场之间的移车费用：$r_1=-2*|a|$；<br>
&emsp;&emsp;$\blacktriangleright$两个租赁场的租车收益：<br>
$$\begin{aligned}
r_2
&=\sum\limits_{n_A=0}^{S_A+a}\sum\limits_{n_B=0}^{S_B-a}\bigl[(\frac{\lambda_{A1}^{n_A}}{n_A!}*e^{-\lambda_{A1}})*(\frac{\lambda_{A2}^{(S_A'-S_A-a+n_A)}}{(S_A'-S_A-a+n_A)!}*e^{-\lambda_{A2}})\\
&*(\frac{\lambda_{B1}^{n_B}}{n_B!}*e^{-\lambda_{B1}})*(\frac{\lambda_{B2}^{(S_B'-S_B-a+n_B)}}{(S_B'-S_B-a+n_B)!}*e^{-\lambda_{B2}})*（n_A+n_B）*10\bigl]；
\end{aligned} $$
&emsp;&emsp;这样，获得的立即奖赏为：$r([s_A,s_B],a,[s'_A,s'_B])=r_1+r_2$。<br>
&emsp;&emsp;(5）折扣因子：$\gamma=0.9$。<br>
&emsp;&emsp;图4.6为关于杰克租车问题的策略迭代过程，经过5轮的策略评估和改进后，得到该问题的最优策略及最优价值，如图4.6（5）、图4.6（6）所示<br>
<center><img src="./image/绘图4.6汽车租赁.jpg" ></center><br>
 &emsp;&emsp;<center>图4.6&ensp;关于杰克租车问题的策略迭代过程</center>
 &emsp;&emsp;该问题需要从一个确定策略出发进行策略迭代。本实验的初始策略为：在任何状态下，不考虑两个租赁场的需求，选择不移动车辆（动作为0）。然后进行策略评估，并根据值函数进行策略改进。如此反复，直到策略收敛，找到最优策略。图4.6（5）即为最优策略图。<br>

In [10]:

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False

matplotlib.use('Agg')

# maximum # of cars in each location
MAX_CARS = 20

# maximum # of cars to move during night
MAX_MOVE_OF_CARS = 5

# expectation for rental requests in first location
# RENTAL_REQUEST_FIRST_LOC = 3
RENTAL_REQUEST_FIRST_LOC = 4
# expectation for rental requests in second location
# RENTAL_REQUEST_SECOND_LOC = 4
RENTAL_REQUEST_SECOND_LOC = 3
# expectation for # of cars returned in first location
# RETURNS_FIRST_LOC = 3
RETURNS_FIRST_LOC = 2
# expectation for # of cars returned in second location
# RETURNS_SECOND_LOC = 2
RETURNS_SECOND_LOC = 3
DISCOUNT = 0.9

# credit earned by a car
RENTAL_CREDIT = 10

# cost of moving a car
MOVE_CAR_COST = 2

# all possible actions
actions = np.arange(-MAX_MOVE_OF_CARS, MAX_MOVE_OF_CARS + 1)

# An up bound for poisson distribution
# If n is greater than this value, then the probability of getting n is truncated to 0
POISSON_UPPER_BOUND = 11

# Probability for poisson distribution
# @lam: lambda should be less than 10 for this function
poisson_cache = dict()


def poisson_probability(n, lam):
    global poisson_cache
    key = n * 10 + lam
    if key not in poisson_cache:
        poisson_cache[key] = poisson.pmf(n, lam)
    return poisson_cache[key]


def expected_return(state, action, state_value, constant_returned_cars):
    """
    @state: [# of cars in first location, # of cars in second location]
    @action: positive if moving cars from first location to second location,
            negative if moving cars from second location to first location
    @stateValue: state value matrix
    @constant_returned_cars:  if set True, model is simplified such that
    the # of cars returned in daytime becomes constant
    rather than a random value from poisson distribution, which will reduce calculation time
    and leave the optimal policy/value state matrix almost the same
    """
    # initailize total return
    returns = 0.0

    # cost for moving cars
    returns -= MOVE_CAR_COST * abs(action)

    # moving cars
    NUM_OF_CARS_FIRST_LOC = min(state[0] - action, MAX_CARS)
    NUM_OF_CARS_SECOND_LOC = min(state[1] + action, MAX_CARS)

    # go through all possible rental requests
    for rental_request_first_loc in range(POISSON_UPPER_BOUND):
        for rental_request_second_loc in range(POISSON_UPPER_BOUND):
            # probability for current combination of rental requests
            prob = poisson_probability(rental_request_first_loc, RENTAL_REQUEST_FIRST_LOC) * \
                poisson_probability(rental_request_second_loc, RENTAL_REQUEST_SECOND_LOC)

            num_of_cars_first_loc = NUM_OF_CARS_FIRST_LOC
            num_of_cars_second_loc = NUM_OF_CARS_SECOND_LOC

            # valid rental requests should be less than actual # of cars
            valid_rental_first_loc = min(num_of_cars_first_loc, rental_request_first_loc)
            valid_rental_second_loc = min(num_of_cars_second_loc, rental_request_second_loc)

            # get credits for renting
            reward = (valid_rental_first_loc + valid_rental_second_loc) * RENTAL_CREDIT
            num_of_cars_first_loc -= valid_rental_first_loc
            num_of_cars_second_loc -= valid_rental_second_loc

            if constant_returned_cars:
                # get returned cars, those cars can be used for renting tomorrow
                returned_cars_first_loc = RETURNS_FIRST_LOC
                returned_cars_second_loc = RETURNS_SECOND_LOC
                num_of_cars_first_loc = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                num_of_cars_second_loc = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                returns += prob * (reward + DISCOUNT * state_value[num_of_cars_first_loc, num_of_cars_second_loc])
            else:
                for returned_cars_first_loc in range(POISSON_UPPER_BOUND):
                    for returned_cars_second_loc in range(POISSON_UPPER_BOUND):
                        prob_return = poisson_probability(
                            returned_cars_first_loc, RETURNS_FIRST_LOC) * poisson_probability(returned_cars_second_loc, RETURNS_SECOND_LOC)
                        num_of_cars_first_loc_ = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                        num_of_cars_second_loc_ = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                        prob_ = prob_return * prob
                        returns += prob_ * (reward + DISCOUNT *
                                            state_value[num_of_cars_first_loc_, num_of_cars_second_loc_])
    return returns


def figure_4_2(constant_returned_cars=True):
    value = np.zeros((MAX_CARS + 1, MAX_CARS + 1))
    policy = np.zeros(value.shape, dtype=np.int)

    iterations = 0
    _, axes = plt.subplots(2, 3, figsize=(40, 20))
    plt.subplots_adjust(wspace=0.1, hspace=0.2)
    axes = axes.flatten()
    while True:
        fig = sns.heatmap(np.flipud(policy), cmap="YlGnBu", vmax=5, vmin=-5,ax=axes[iterations],center=0)
        fig.set_ylabel('# B ',fontsize=30)
        fig.set_yticks(list(reversed(range(MAX_CARS + 1))))
        fig.set_xlabel('# A', fontsize=30)
        fig.set_title('policy {}'.format(iterations), fontsize=30)

        # policy evaluation (in-place)
        while True:
            old_value = value.copy()
            for i in range(MAX_CARS + 1):
                for j in range(MAX_CARS + 1):
                    new_state_value = expected_return([i, j], policy[i, j], value, constant_returned_cars)
                    value[i, j] = new_state_value
            max_value_change = abs(old_value - value).max()
            print('max value change {}'.format(max_value_change))
            if max_value_change < 1e-4:
                break

        # policy improvement
        policy_stable = True
        for i in range(MAX_CARS + 1):
            for j in range(MAX_CARS + 1):
                old_action = policy[i, j]
                action_returns = []
                for action in actions:
                    if (0 <= action <= i) or (-j <= action <= 0):
                    # if(-i <= action <= 0) or (0 <= action <= j):
                        action_returns.append(expected_return([i, j], action, value, constant_returned_cars))
                    else:
                        action_returns.append(-np.inf)
                new_action = actions[np.argmax(action_returns)]
                policy[i, j] = new_action
                if policy_stable and old_action != new_action:
                    policy_stable = False
        print('policy stable {}'.format(policy_stable))

        if policy_stable:
            fig = sns.heatmap(np.flipud(value), cmap="YlGnBu", ax=axes[-1])
            fig.set_ylabel('# 第一地点的车辆数', fontsize=30)
            fig.set_yticks(list(reversed(range(MAX_CARS + 1))))
            fig.set_xlabel('# 第二地点的车辆数', fontsize=30)
            fig.set_title('optimal value', fontsize=30)
            break

        iterations += 1

    plt.savefig('images/figure_4_2.png')
    plt.close()


if __name__ == '__main__':
    figure_4_2()

max value change 242.69734618420745
max value change 124.0988199892968
max value change 70.32718327075344
max value change 48.94488247898738
max value change 41.488250014681086
max value change 35.03521693218764
max value change 29.64178677090021
max value change 25.8308384863233
max value change 22.396739990982212
max value change 19.350456685914367
max value change 16.674507474144264
max value change 14.339324865231788
max value change 12.311095840284509
max value change 10.555662149529894
max value change 9.040461785396815
max value change 7.73544365819572
max value change 6.613426120390784
max value change 5.650150793260025
max value change 4.824171763458878
max value change 4.116660390788297
max value change 3.5111723929849177
max value change 2.9934043848315355
max value change 2.5509554550628764
max value change 2.173102354529874
max value change 1.850592597168145
max value change 1.5754571756690439
max value change 1.340843032431053
max value change 1.1408645166079623
max value

policy stable False
max value change 0.039185786635698605
max value change 0.010393260391310832
max value change 0.004017504204284705
max value change 0.0019959310058084156
max value change 0.001017128579974269
max value change 0.0005309162257844946
max value change 0.0002695564863870459
max value change 0.00013530335172617924
max value change 6.822847399234888e-05
policy stable True


#### 4.1.3.2 基于动作值函数的策略迭代
&emsp;&emsp;与基于状态值函数的策略迭代不同，基于动作值函数的策略迭代是在当前策略$\pi(a|s)$下，利用式（4.2）对状态-动作对进行评估。具体如算法4.4所示。
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.4</b>基于随机MDP的动作值函数$q_\pi$策略迭代算法</font>
<br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
&emsp;<font size=3.5>初始策略$\pi(a|s)$，动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in{S_s}$，$a\in{\mathcal{A(s)}}$，初始化动作值函数：如$Q(s,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实值</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>策略评估：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略评估$l=1,2,\cdots$ </font><br>
<font size=3.5><b>策</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;在当前策略$\pi(a|s)$下，通过算法4.2策略评估迭代，得到动作值函数$q_\pi{:}Q(s,a)$： </font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>策略改进：</b></font>
<font size=3.5>5.&emsp;&emsp;&emsp;$policy$_$stable$←<b>True</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;<b>for</b>&emsp;每个状态$s$&emsp;<b>do</b></font><br>
<font size=3.5><b>略</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;
$old$_$policy←\pi(a|s)$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;
$max$_$action←\mathop{\arg\max}\limits_{a}Q(s,a);$&emsp;$max$_$conut←count(max(Q(s,a)))$
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>for</b>&emsp;每个状态-动作对$(s,a')$&emsp;<b>do</b>
</font><br>
<font size=3.5><b>迭</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>if</b>&emsp;$a' == max$_$action$&emsp;<b>then</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=\frac{1}{(max\_count)}$</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>else</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=0$</b>
</font><br>
<font size=3.5><b>代</b></font>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end&emsp;if</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>16.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>if</b>&emsp;$old$_$policy\neq\pi(a|s)$&emsp;<b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>17.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$policy$-$stable$←<b>False</b></font><br>
</font>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>18.&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>19.&emsp;
<b>until&emsp;</b>$policy$_$stable$=<b>True</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;<font size=3.5>$q_*=Q,\pi_*=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>
&emsp;&emsp;算法4.4与算法4.3的区别在于：算法4.3是基于状态值函数的策略迭代，利用$V(s)$值函数和转移动态，计算该状态$s$下所有动作$a$的动作值$Q(s,a)$，然后根据$Q(s,a)$，选择较优的策略，再进行改进。而算法4.2是基于动作值函数$Q(s,a)$进行的策略迭代，因为在$Q(s,a)$中包含了关于动作的信息，因此较优策略可以根据$Q(s,a)$直接得到。<br>
&emsp;&emsp;<b>例4.7</b> 将基于动作值函数的策略迭代算法4.4应用于例4.1的确定环境扫地机器人任务中，经过多轮迭代后，得到表4.4所示的动作值函数和策略迭代更新过程。<br>

In [11]:
def Policy_Iterration_byQ():
    initPolicy()
    k = 1
    while True:  # Evaluate the current policy
        print("迭代的次数",k)
        Q = policy_eval()
        print(Q)
        Q,policy_stable = policy_improvement(Q,policy)
        print("*" * 100)
        k+=1
        if policy_stable:  # # If the policy is stable we've found an optimal policy. Return it
            return policy, Q
Policy_Iterration_byQ()

迭代的次数 1
[-1.1105499  -1.3594703  -1.61520774 -0.3289769   1.36840924 -1.41690492
 -2.37225627 -4.368582   -0.98686494  0.         -1.83058844 -4.71632499
  0.         -3.98676594 -0.29972245 -0.73147825 -2.16245658 -4.64868148
 -2.16047706 -0.88719323  0.         -0.7157996  -1.77179337 -1.27974528
 -0.86677528]
Q[0][0]  -0.585183 = 0 + 0.8 * -0.73:
Q[0][3]  -0.572640 = 0 + 0.8 * -0.72:
Q[1][0]  -1.729965 = 0 + 0.8 * -2.16:
Q[1][2]  1.000000 = 1 + 0.8 * 0.00:
Q[1][3]  -1.417435 = 0 + 0.8 * -1.77:
Q[2][0]  -3.718945 = 0 + 0.8 * -4.65:
Q[2][2]  -0.572640 = 0 + 0.8 * -0.72:
Q[2][3]  -1.023796 = 0 + 0.8 * -1.28:
Q[3][0]  -1.728382 = 0 + 0.8 * -2.16:
Q[3][2]  -1.417435 = 0 + 0.8 * -1.77:
Q[3][3]  -0.693420 = 0 + 0.8 * -0.87:
Q[4][0]  -0.709755 = 0 + 0.8 * -0.89:
Q[4][2]  -1.023796 = 0 + 0.8 * -1.28:
Q[5][0]  -1.464471 = 0 + 0.8 * -1.83:
Q[5][1]  1.000000 = 1 + 0.8 * 0.00:
Q[5][3]  -1.729965 = 0 + 0.8 * -2.16:
Q[6][0]  -3.773060 = 0 + 0.8 * -4.72:
Q[6][1]  -0.572640 = 0 + 0.8 * -0.72:
Q[6][2

(array([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]),
 array([1.2288, 1.536 , 1.92  , 2.4   , 3.    , 1.536 , 1.92  , 2.4   ,
        3.    , 0.    , 1.2288, 1.536 , 0.    , 2.4   , 3.    , 1.    ,
        1.2288, 1.536 , 1.92  , 2.4   , 0.    , 1.    , 1.2288, 1.536 ,
        1.92  ]))

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.4 面向确定环境扫地机器人任务的基于动作值函数的策略迭代更新过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    
  
    
   <tr>
        <td><p style="text-align:center">$\pi_0$</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.33;0.00;0.33;0.33</p></td>
        <td><p style="text-align:center">0.25;0.25;0.25;0.25</p></td>
        <td><p style="text-align:center">0.00;0.5;0.5;0.00</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">-1.73;×;1.00;-1.42</p></td>
        <td><p style="text-align:center">-3.72;×;-0.57;-1.02</p></td>
        <td><p style="text-align:center">-1.73;×;-1.42;-0.69</p></td>
        <td><p style="text-align:center">-10.00;-1.42;-1.73;-1.73</p></td>
        <td><p style="text-align:center">×;3.00;-0.26;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_1$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$Q_2$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_2$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
       <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_4$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
   
   <tr>
        <td><p style="text-align:center">$Q_5$</p></td>
        <td><p style="text-align:center">0.98;×;1.00;0.98</p></td>
        <td><p style="text-align:center">1.23;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.98;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.98;0.98;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_5$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
 
   <tr>
        <td><p style="text-align:center">$Q_6$</p></td>
        <td><p style="text-align:center">0.98;×;1.00;0.98</p></td>
        <td><p style="text-align:center">1.23;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.98;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.98;0.98;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_6$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_*$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
</table>

## 4.2 值迭代
&emsp;&emsp;在策略迭代中，每轮策略改进之前都涉及策略评估，每次策略评估都需要多次遍历才能保证状态值函数在一定程度上得到收敛，这将消耗大量的时间和计算资源。从例4.1中可以看出，当$\tau\geq4$ 时，值函数的变化对策略不再有影响。于是根据迭代次数与策略稳定的相互关系，考虑在单步评估之后就进入改进过程，即采取截断式策略评估，在一次遍历完所有的状态后立即停止策略评估，进行策略改进，这种方法称为值迭代。基于状态值函数的值迭代公式为：<br>
$$\begin{aligned}
v_\iota(s)&=\max_a\mathbb{E}_\pi（R_{t+1}+\gamma{v_\pi(S_{t+1})}|S_t=s,A_t=a)\\
&=\max_a\sum_{s',r}p（s',r|s,a）[r+\gamma{v_{\iota-1}(s')}]
\end{aligned}\tag{4.10} $$
&emsp;&emsp;该式与策略评估方程（4.1）的唯一区别在于：对状态值函数进行更新时，仅使用最大动作值函数，而非所有动作值函数的期望。可以证明：当$v_*$存在时，对任意初始$v_0$，序列$\{v_\iota\}$一定能收敛到$v_*$。<br>
&emsp;&emsp;值迭代将贝尔曼最优方程变成一条更新规则，其状态值函数的计算过程仅体现了状态转移的随机性，而与策略无关。理论上值迭代依然需要迭代无数次才能收敛，但与策略迭代一样，如果一次遍历后值函数的变化低于阈值，可以提前停止。由于策略迭代的收敛速度更快一些，因此在状态空间较小时，最好选用策略迭代方法。当状态空间较大时，值迭代的计算量更小些。<br>
&emsp;&emsp;算法4.5给出了在有穷状态空间MDP中，基于状态值函数的值迭代算法。
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.5</b> 基于状态值函数的值迭代算法</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
<font size=3.5>动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in{S_s}$，初始化状态值函数：如$V(s)$=0</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实值</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>评估过程：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略评估$l=1,2,\cdots$ </font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;$delta←0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;<b>for</b>&emsp;每个状态$s$&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;$v←V(s)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;$V(s)←\max\limits_a\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{V(s')}]$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;$delta←max(delta,|v-V(s)|)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;<b>end &emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;<b>until&emsp;$delta<\theta$</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>最优策略：</b></font>
<font size=3.5>11.&emsp;<b>for</b>&emsp;每个状态$s$ &emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;$max$_$action←\mathop{\arg\max}\limits_{a}\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{V(s')}]$</font><br>    
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;$max$_$count$←$conut(max$_$action)$</font><br> 
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;
<b>for</b>&emsp;每个状态-动作对$(s,a')$&emsp;<b>do</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;
<b>if</b>&emsp;$a' == max$_$action$&emsp;<b>then</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>16.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=\frac{1}{(max\_count)}$</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>17.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>else</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>18.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=0$</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>19.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end&emsp;if</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>20.&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>21.&emsp;
<b>end&emsp;for</b></font><br>

<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;&emsp;<font size=3.5>$v_*=V,\pi_*=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>
&emsp;&emsp;与策略迭代类似，也可以直接基于动作值函数进行值迭代，迭代公式为：
$$\begin{aligned}
Q(s,a)=\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{\max_{a'}}Q(s',a')]
\end{aligned} \tag{4.11}$$
&emsp;&emsp;算法4.6给出了在有穷状态空间MDP中，基于动作值函数的值迭代算法。<br>






<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法4.6</b> 基于动作值函数的值迭代算法</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 入：</b></font>
<font size=3.5>动态性$p$，奖赏函数$r$，折扣因子$\gamma$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in{S_s}$，$a\in{\mathcal{A(s)}}$，初始化状态值函数：如$Q(s,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;阈值$\theta$设置为一个较小的实值</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>评估过程：</b></font>
<font size=3.5>3.&emsp;<b>repeat</b> 对每一轮策略迭代$l=1,2,\cdots$ </font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;$delta←0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;<b>for</b>&emsp;每个状态-动作对$(s,a)$&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;$q←Q(s,a)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;$Q(s,a)=\sum\limits_{s',r}p(s',r|s,a)[r+\gamma{\max\limits_{a'}Q(s',a')}]$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;$delta←max(delta,|q-Q(s,a)|)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;<b>until &emsp;$\Delta<\theta$</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>最优策略：</b></font>
<font size=3.5>10.&emsp;<b>for</b>&emsp;每个状态$s$ &emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;$max$_$action←\mathop{\arg\max}\limits_{a}Q(s,a);max$_$count←count(max(Q(s,a)))$</font><br>    
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;
<b>for</b>&emsp;每个状态-动作对$(s,a')$&emsp;<b>do</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;
<b>if</b>&emsp;$a' == max$_$action$&emsp;<b>then</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=\frac{1}{(max\_count)}$</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>else</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>16.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
$\pi(a'|s)=0$</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>17.&emsp;&emsp;&emsp;&emsp;&emsp;
<b>end&emsp;if</b>
</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>18.&emsp;&emsp;&emsp;
<b>end&emsp;for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>19.&emsp;
<b>end&emsp;for</b></font><br>

<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;&emsp;<font size=3.5>$q_*=Q,\pi_*=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>
&emsp;&emsp;算法4.5和算法4.6既可用于确定环境MDP又可用于随机环境MDP。<br>
&emsp;&emsp;<b>例4.8</b> 将基于状态值函数的值迭代算法4.5应用于例4.1的确定环境扫地机器人任务中，可以得到以下状态值迭代更新过程：<br>
（1）当$l=$0时（$l$为值迭代的次数），对于所有的$s$初始化为：$V(s)=$0；<br>
（2）当$l=$1时，以状态$S_{24}$为例。在策略$\pi$下，只能采取向下和向左2个动作，概率各为0.5。采取向下的动作时，到达状态$S_{19}$($V(S_{19}$)=0)，并可以捡到垃圾，获得$r_1=$+3的奖赏；采取向左的动作时，到达状态$S_{23}$($V(S_{23}$)=2.40)，获得$r_2=$0的奖赏。根据算法4.2计算得：<br>
&emsp;&emsp;$\begin{aligned}
V(S_{24})&=max(r_1+\gamma{V(S_{19})},r_2+\gamma{V(S_{23})})\\
&=max(3+0.8*0,0+0.8*2.40)\\
&=max(3,1.92)\\
&=3.00\\
\end{aligned}$
<br>
&emsp;&emsp;同理可以计算状态$S_5$和$S_6$的状态值函数：<br>
&emsp;&emsp;$\begin{aligned}
V(S_5)&=max(r_1+\gamma{V(S_{10})},r_2+\gamma{V(S_{0})},r_3+\gamma{V(S_{6})})\\
&=max(0+0.8*0,1+0.8*0,0+0.8*0)\\
&=1.00
\end{aligned}$
<br>
&emsp;&emsp;$\begin{aligned}
V(S_{6})&=max(r_1+\gamma{V(S_{11})},r_2+\gamma{V(S_{1})},r_3+\gamma{V(S_{5})},r_4+\gamma{V(S_{7})})\\
&=max(0+0.8*0,0+0.8*1.00,0+0.8*1.00,0+0.8*0)\\
&=0.80
\end{aligned}$<br>
&emsp;&emsp;按顺序计算完一轮后，得到值函数$V(s)$，如图4.7（$\iota$=1）所示。
&emsp;&emsp;当$l=$2时，以状态$S_{22}、S_{23}、S_{24}$为例，计算状态值函数。异步计算方式，通常与迭代的计算顺序有关，根据例4.1规定，在每一轮次中，这3个状态的计算顺序为$S_{22}、S_{23}、S_{24}$。

&emsp;&emsp;$\begin{aligned}
V(S_{22})&=max(r_1+\gamma{V(S_{17})},r_2+\gamma{V(S_{21})},r_3+\gamma{V(S_{23})})\\
&=max(0+0.8*2.40,0+0.8*0.41,0+0.8*2.40)\\
&=max(1.92,0.33,1.92)\\
&=1.92
\end{aligned}$<br>

&emsp;&emsp;$\begin{aligned}
V(S_{23})&=max(r_1+\gamma{V(S_{18})},r_2+\gamma{V(S_{22})},r_3+\gamma{V(S_{24})})\\
&=max(0+0.8*3.00,0+0.8*1.92,0+0.8*3.00)\\
&=max(2.40,1.54,2.40)\\
&=2.40
\end{aligned}$<br>

&emsp;&emsp;$\begin{aligned}
V(S_{24})&=max(r_1+\gamma{V(S_{19})},r_2+\gamma{V(S_{23})})\\
&=max(3+0.8*0.00,0+0.8*2.40)\\
&=3.00
\end{aligned}$<br>
&emsp;&emsp;按顺序计算完一轮后，得到值函数V(s)，如图4.7（$\iota=2$）所示。<br>
&emsp;&emsp;（4）当$\iota=6$时，$|v_l(s)-v_{l-1}(s)|_∞<\theta$，认为$v_l(s)$已经收敛于$v_*（s）$，计算得到的$v_*（s）$就是最优状态值函数。<br>
&emsp;&emsp;图4.7为确定环境扫地机器人任务通过状态值迭代得到的结果<br>。
<center><img src="./image/图4-7.jpg"></center>
<center>图4.7&ensp;面向确定环境扫地机器人任务通过状态值迭代得到的结果</center>


In [12]:
# 更新状态值函数
def value_update_byIter(s,numb):  # 传入当前状态
    value_new = 0
    if s in [9,20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
        pass
    else:
        Q =[]
        successor = getsuccessor(s)  # 得到所有可能的下一状态list
        # rewards = reward(s)  # 得到当前状态的奖励
        # print("s %s="%s,end="")
        for next_state in successor:  # 遍历所有可能的下一状态
            rewards = reward(next_state)
            value_new = rewards + gamma * value[next_state]  # 计算公式，得到当前状态的状态价值函数
            Q.append(value_new)
            # print("%.2f*(%d+%.1f*%.2f) = %.2f"%(1/len(successor),rewards,gamma,value[next_state],value_new))
            # 注意前面的1/len(successor)为该s状态能够到下个状态的个数概率，该代码是第一次迭代时的固定策略π(a|s)
        value_new = max(Q)
        # print("第%d个状态最大值：%.2f"%(numb,value_new))
        # print()
    return value_new
def initial_state():
    v = np.array(value).reshape(world_h, world_w)  # 调整初始化状态值函数矩阵
    print(v)
def Caculate_Q(s, V, num,discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    # Q[:][:] = -1000
    for a in ['w', 'e', 'n', 's']:  # 遍历能走的动作
        # for prob, next_state, reward, done in env.P[s][a]:
        #     Q[s][a] += prob * (reward + discount_factor * V[next_state])  # 计算当前状态s下的动作值函数列表 [q1,q2,q3,q4]
        next_state = next_states(s,a)
        if next_state == s: #碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        Q[s][numberA]= rewards + discount_factor * V[next_state]
        print("Q[%s][%d]  %.2f = %s + 0.8 * %.2f:"%(num,numberA,Q[s][numberA],rewards,V[next_state]))
        
def main():
    max_iter = 7  # 最大迭代次数
    initial_state()

    iter = 1

    while iter < max_iter:
        numb = -1
        for s in [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]:  # 遍历所有状态
            numb += 1
            value[s] = value_update_byIter(s,numb)  # 更新状态值函数
            Caculate_Q(s,value,numb)
        v = np.array(value).reshape(world_h, world_w)  # 更新状态值函数矩阵

        if (iter <= 10) or (iter % 10 == 0):  # 前1次 + 每10次打印一次
            print('k=', iter)  # 打印迭代次数
            print(np.round(v, decimals=4))  # np.round() 返回浮点数的四舍五入值

        iter += 1
        
main()
    

[[-0.79536914 -1.02180937 -1.26552842 -0.15644465  1.36897348]
 [-1.06603538 -1.96139755 -3.88931472 -0.74546696  0.        ]
 [-1.43462882 -4.17595855  0.         -3.56312877 -0.05632986]
 [-0.48899798 -1.79038799 -4.12515305 -1.78910237 -0.61184602]
 [ 0.         -0.4777829  -1.39174745 -0.96110887 -0.59922071]]
Q[0][3]  -0.38 = 0 + 0.8 * -0.48:
Q[0][0]  -0.39 = 0 + 0.8 * -0.49:
Q[1][2]  1.00 = 1 + 0.8 * 0.00:
Q[1][3]  -1.11 = 0 + 0.8 * -1.39:
Q[1][0]  -1.43 = 0 + 0.8 * -1.79:
Q[2][2]  0.80 = 0 + 0.8 * 1.00:
Q[2][3]  -0.77 = 0 + 0.8 * -0.96:
Q[2][0]  -3.30 = 0 + 0.8 * -4.13:
Q[3][2]  0.64 = 0 + 0.8 * 0.80:
Q[3][3]  -0.48 = 0 + 0.8 * -0.60:
Q[3][0]  -1.43 = 0 + 0.8 * -1.79:
Q[4][2]  0.51 = 0 + 0.8 * 0.64:
Q[4][0]  -0.49 = 0 + 0.8 * -0.61:
Q[5][3]  -1.43 = 0 + 0.8 * -1.79:
Q[5][0]  -1.15 = 0 + 0.8 * -1.43:
Q[5][1]  1.00 = 1 + 0.8 * 0.00:
Q[6][2]  0.80 = 0 + 0.8 * 1.00:
Q[6][3]  -3.30 = 0 + 0.8 * -4.13:
Q[6][0]  -3.34 = 0 + 0.8 * -4.18:
Q[6][1]  0.80 = 0 + 0.8 * 1.00:
Q[7][2]  0.64 = 0 

&emsp;&emsp;表4.5和表4.6分别给出了通过异步计算方式得到的，基于状态值函数和动作值函数的扫地机器人任务的值迭代更新过程。<br>
<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.5 面向确定环境扫地机器人任务的基于状态值函数的值迭代更新过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    
  <tr>
        <td><p style="text-align:center">$v_0$</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
        <td><p style="text-align:center">0.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$v_1$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">0.80</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$v_2$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">0.80</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$v_3$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">0.80</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">0.64</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
      <tr>
        <td><p style="text-align:center">$v_4$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">0.80</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$v_5$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">1.23</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$v_6$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">1.23</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr> 
    <tr>
        <td><p style="text-align:center">$v_*$</p></td>
        <td><p style="text-align:center">1.00</p></td>
        <td><p style="text-align:center">1.23</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">1.54</p></td>
        <td><p style="text-align:center">3.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_*$</p></td>
        <td><p style="text-align:center">0.98;×;1.00;0.98</p></td>
        <td><p style="text-align:center">1.23;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.98;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.98;0.98;1.54</p></td>
        <td><p style="text-align:center">×;3.00;1.92;0.00</p></td>
    </tr>
   <tr>
        <td><p style="text-align:center">$\pi_*$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
</table>
<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=3.5><center>表4.6 面向确定扫地机器人任务的基于动作值函数的值迭代更新过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_1$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$S_{3}$</p></td>
        <th><p style="text-align:center">$S_{7}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    
  <tr>
        <td><p style="text-align:center">$Q_0$</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.00;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">×;0.00; 0.00;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">0.00;×;1.00;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.80;0.00</p></td>
        <td><p style="text-align:center">0.00;×;0.64;0.00</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$Q_2$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;0.51</p></td>
        <td><p style="text-align:center">0.41;×;0.64;0.41</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.41</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$Q_3$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;0.51</p></td>
        <td><p style="text-align:center">0.41;×;0.64;0.41</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_4$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;0.51</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_5$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$Q_6$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$Q_*$</p></td>
        <td><p style="text-align:center">0.64;×;1.00;0.64</p></td>
        <td><p style="text-align:center">0.51;×;0.80;1.23</p></td>
        <td><p style="text-align:center">1.54;×;0.64;1.54</p></td>
        <td><p style="text-align:center">-10.00;0.64;0.64;0.00</p></td>
        <td><p style="text-align:center">×;3.00; 1.92;×</p></td>
    </tr>
    
   <tr>
        <td><p style="text-align:center">$\pi_*$</p></td>
        <td><p style="text-align:center">0.00;0.00;1.00;0.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">1.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
</table>
&emsp;&emsp;虽然策略迭代、值迭代等DP方法可以有效解决强化学习问题，但它仍然存在较多缺点：<br>
&emsp;&emsp;（1）在进行最优策略计算时，必须知道状态转移概率$p$。<br>
&emsp;&emsp;（2）DP的推演是整个树状展开的，计算量大，存储消耗资源多。<br>
&emsp;&emsp;（3）每次回溯，所有可能的下一状态和相应动作都要被考虑在内，存在维度灾难问题。<br>
&emsp;&emsp;（4）由于策略初始化的随机性，不合理的策略可能会导致算法无法收敛。<br>
&emsp;&emsp;<b>例4.9</b> 赌徒问题 一个赌徒投掷一个骰子来累加骰子点数之和。赌徒可以选择重新投掷骰子或者结束整局游戏。如果选择结束整局游戏骰子总和数刚好18点则赌徒赢得10元，骰子点数总和超过了18则输掉（骰子点数总数-18）的资金，少于18则输掉（$\frac{18-骰子点数总数}{2}$&emsp;&emsp;）的资金。当点数超过或者等于18时，会自动结束整局游戏。<br>
&emsp;&emsp;利用值迭代方法，在折扣率$\gamma$=0.9时，计算赌徒在不同状态最优策略。<br>
&emsp;&emsp;该问题的MDP模型为:<br>
&emsp;&emsp;（1）状态空间：当前赌徒共扔掷了骰子点数的总和，即$s$=$[0,1,2,3,4,5...,10,...,23]$共 24个状态。<br>
&emsp;&emsp;（2）动作空间：赌徒可以选择重新投掷骰子或是结束整局比赛，即两个动作$A(s)$={0,1}0代表结束游戏，1代表掷骰子。<br>
&emsp;&emsp;（3）状态转移：即在当前状态$s$下，采取动作$a$，到达下一状态$s'$的概率。假设$s$=1时执行了动作1。那么$s'$有可能的状态是{2,3,4,5,6,7}共6种状态，每个状态的概率为$\frac{1}{6}$。但当$s\geq36$，会自动执行动作0来结束整个回合并获得回报。故状态转移函数可写成：<br>
$$
p(s,a,s')=\begin{cases}
\frac{1}{6},a=1,s\neq[18,23]且s'\in[s+1,s+6]\\
1,a=0,s=s'\\
0,其他\\
 \end{cases} $$
&emsp;&emsp;（4）立即奖赏：赌徒重新投掷筛子会获得0的立即奖赏。当整局比赛结束，立即回报<br>
$$
r=\begin{cases}
-(s-18),s>18\\
+10,s=18\\
-\frac{(18-s)}{2},s<18\\
 \end{cases} $$
 &emsp;&emsp;（5）折扣因子：$\gamma$=0.9。<br>
 

## 4.3 广义策略迭代
&emsp;&emsp;在策略迭代算法中，策略评估与策略改进两个流程交替进行，且每个流程都在另一个开始前完成，这样的方法被称为经典策略迭代（classical policy iteration）。广义策略迭代（Generalized Policy Iteration，GPI）则体现了策略评估与策略改进交替进行的一般性，强调策略评估和策略改进的交互关系，而不关心策略评估到底迭代了多少次，或具体的策略评估和策略改进的细节。在GPI中，策略评估没结束，就可以进行策略改进，只要这两个过程都能不断地更新，就能收敛到最优值函数和最优策略。从这一角度看，值迭代也属于GPI，而实际上几乎所有的强化学习方法都可以被描述为GPI。<br>
&emsp;&emsp;GPI体现了评估和改进之间相互竞争与合作的关系：基于贪心策略，使得值函数与当前策略不匹配，而保持值函数与策略一致就无法更新策略。在长期的博弈后，两个流程会趋于一个目标，即最优值函数和最优策略。<br>
&emsp;&emsp;策略总是基于特定的值函数进行改进的，值函数始终会收敛于对应的特定策略的真实值函数，当评估和改进都稳定时，贝尔曼最优方程便可成立，此时得到最优值函数和最优策略。换句话说，值函数只有与当前策略一致时才稳定，且策略只有是当前值函数的贪心策略时才稳定。<br>
## 小结
&emsp;&emsp;在环境已知的前提下，基于马尔可夫决策过程，动态规划可以很好的完成强化学习任务。策略评估通常对于给定的策略，不断迭代计算每个状态（或状态-动作对）的价值。其迭代方法主要是利用对后继状态（或状态-动作对）价值的估计，来更新当前状态（或状态-动作对）价值的估计，也就是用自举的方法。策略改进是采用贪心算法，利用动作值函数获得更优的策略，每次都选择最好的动作。策略迭代是重复策略评估和策略改进的迭代，直到策略收敛，找到最优的策略。但是策略迭代需要多次使用策略评估才能得到收敛的状态（或状态-动作对）值函数，即策略评估是迭代进行的，只有在$v_\pi$收敛时，才能停止迭代。值迭代无需等到其完全收敛，提早的计算出贪心策略，截断策略评估，在一次遍历后即刻停止策略评估，并对每个状态进行更新。实践证明，值迭代算法收敛速度优于策略迭代算法。广义策略迭代则体现了策略评估与策略改进交替进行的一般性。在GPI中，策略评估和策略改进同时进行，只要这两个过程都能不断地更新，就能收敛到最优值函数和最优策略。从这一角度看，值迭代也属于GPI，而实际上几乎所有的强化学习方法都可以被描述为GPI。